---
description: parameterset list
output-file: parametersets.html
title: parametersets

---

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp parametersets

In [ ]:
#| hide

from hwpapi.core import App

app = App()


In [ ]:
#| export
from __future__ import annotations
from hwpapi.functions import from_hwpunit, to_hwpunit, convert_hwp_color_to_hex, convert_to_hwp_color
import pprint


class ParameterSet:
    """
    Base class for HWP parameter sets.
    
    Provides a unified interface for working with HWP parameter objects,
    handling both COM objects with SetID and regular Python objects.
    """

    def __init__(self, parameterset):
        """Initialize with a parameter set object."""
        self._pset = parameterset
        self._is_pset = hasattr(parameterset, "SetID")
        self.attributes_names = []

    @property
    def parameterset(self):
        """Get the underlying parameter set object."""
        return self._pset

    def _get_value(self, name):
        """Get a value from the parameter set."""
        if self._is_pset:
            return self._pset.Item(name)
        return getattr(self._pset, name)
    
    def _set_value(self, name, value):
        """Set a value in the parameter set."""
        if self._is_pset:
            return self._pset.SetItem(name, value)
        return setattr(self._pset, name, value)

    def _del_value(self, name):
        """Delete a value from the parameter set."""
        if self._is_pset:
            return self._pset.RemoveItem(name)
        return False
    
    def update_from(self, pset):
        """Update this parameter set with values from another parameter set."""
        for key in self.attributes_names:
            value = getattr(pset, key, None)
            
            if isinstance(value, ParameterSet):
                # Recursively update nested parameter sets
                target = getattr(self, key)
                target.update_from(value)
            elif value is None:
                # Remove the attribute if value is None
                self._del_value(key)
            elif value:
                # Set the attribute if value is truthy
                setattr(self, key, value)
        return self

    def serialize(self):
        """Convert the parameter set to a dictionary."""
        result = {}
        for key in self.attributes_names:
            value = getattr(self, key, None)
            if isinstance(value, ParameterSet):
                value = value.serialize()
            result[key] = value
        return result
    
    def __str__(self):
        """String representation of the parameter set."""
        data = {
            "name": self.__class__.__name__, 
            "values": self.serialize()
        }
        return pprint.pformat(data, indent=4, width=60)
    
    def __repr__(self):
        """Representation of the parameter set."""
        return self.__str__()

    @staticmethod        
    def _typed_prop(key, doc, expected_type):
        """Create a property for typed parameter sets."""
        def getter(self):
            value = self._get_value(key)
            if value is None:
                return None
            # Wrap in expected type if not already wrapped
            if not isinstance(value, expected_type()):
                return expected_type()(value)
            return value
        
        def setter(self, pset):
            if pset is None:
                return self._del_value(key)
            
            # Validate type
            if not isinstance(pset, expected_type()):
                raise TypeError(f"Value for '{key}' must be of type {expected_type.__name__}")
            
            # Get or create target
            target = self._get_value(key)
            if not target:
                target = self._pset.CreateItemSet(key, expected_type().SetID)
            
            # Update target with new values
            expected_type()(target).update_from(pset)
        
        return property(getter, setter, doc=doc)    

    @staticmethod      
    def _int_prop(key, doc, min_val=None, max_val=None):
        """Create a property for integer values with optional range validation."""
        def getter(self):
            return self._get_value(key)
        
        def setter(self, value):
            if value is None:
                return self._del_value(key)
            
            # Type validation
            if not isinstance(value, int):
                raise TypeError(f"Value for '{key}' must be an integer, got {type(value).__name__}")
            
            # Range validation
            if min_val is not None and value < min_val:
                raise ValueError(f"Value for '{key}' must be >= {min_val}, got {value}")
            if max_val is not None and value > max_val:
                raise ValueError(f"Value for '{key}' must be <= {max_val}, got {value}")
            
            return self._set_value(key, value)
        
        return property(getter, setter, doc=doc)
    
    @staticmethod 
    def _bool_prop(key, doc):
        """Create a property for boolean values (0 or 1)."""
        def getter(self):
            return self._get_value(key)
        
        def setter(self, value):
            if value is None:
                return self._del_value(key)
            
            # Convert boolean to 0/1 if needed
            if isinstance(value, bool):
                value = int(value)
            
            # Validate value is 0 or 1
            if value not in [0, 1]:
                raise ValueError(f"Value for '{key}' must be 0 or 1, got {value}")
            
            return self._set_value(key, value)
        
        return property(getter, setter, doc=doc)
    
    @staticmethod 
    def _color_prop(key, doc):
        """Create a property for color values with hex conversion."""
        def getter(self):
            hwp_color = self._get_value(key)
            return convert_hwp_color_to_hex(hwp_color)
        
        def setter(self, value):
            if value is None:
                return self._del_value(key)
            
            hwp_color = convert_to_hwp_color(value)
            return self._set_value(key, hwp_color)
        
        return property(getter, setter, doc=doc)
    
    @staticmethod 
    def _unit_prop(key, unit, doc):
        """Create a property for unit-based values with automatic conversion."""
        def getter(self):
            hwp_value = self._get_value(key)
            return from_hwpunit(hwp_value, unit)
        
        def setter(self, value):
            if value is None:
                return self._del_value(key)
            
            # Type validation
            if not isinstance(value, (int, float)):
                raise TypeError(f"Value for '{key}' must be a number, got {type(value).__name__}")
            
            # Convert to HWP unit
            hwp_value = to_hwpunit(float(value), unit)
            return self._set_value(key, hwp_value)
        
        return property(getter, setter, doc=doc)
    
    @staticmethod 
    def _mapped_prop(key, mapping, doc):
        """Create a property for mapped values (string <-> integer)."""
        reverse_mapping = {v: k for k, v in mapping.items()}

        def getter(self):
            numeric_value = self._get_value(key)
            # Return the string representation if available, otherwise the numeric value
            return reverse_mapping.get(numeric_value, numeric_value)
        
        def setter(self, value):
            if value is None:
                return self._del_value(key)
            
            # Handle string input
            if isinstance(value, str):
                if value not in mapping:
                    valid_options = list(mapping.keys())
                    raise ValueError(f"Invalid value '{value}' for '{key}'. Valid options: {valid_options}")
                numeric_value = mapping[value]
            
            # Handle integer input
            elif isinstance(value, int):
                if value not in reverse_mapping:
                    valid_options = list(reverse_mapping.keys())
                    raise ValueError(f"Invalid value {value} for '{key}'. Valid options: {valid_options}")
                numeric_value = value
            
            # Invalid type
            else:
                raise TypeError(f"Value for '{key}' must be string or integer, got {type(value).__name__}")
            
            return self._set_value(key, numeric_value)
        
        return property(getter, setter, doc=doc)


    # --- Additional property types ---
    @staticmethod
    def _str_prop(key, doc):
        """Create a property for string values."""
        def getter(self):
            return self._get_value(key)
        
        def setter(self, value):
            if value is None:
                return self._del_value(key)
            
            if not isinstance(value, str):
                raise TypeError(f"Value for '{key}' must be a string, got {type(value).__name__}")
            
            return self._set_value(key, value)
        
        return property(getter, setter, doc=doc)

    @staticmethod
    def _int_list_prop(key, doc):
        """Create a property for lists of integers."""
        def getter(self):
            return self._get_value(key)
        
        def setter(self, value):
            if value is None:
                return self._del_value(key)
            
            # Validate list of integers
            if not isinstance(value, list):
                raise TypeError(f"Value for '{key}' must be a list, got {type(value).__name__}")
            
            if not all(isinstance(item, int) for item in value):
                raise TypeError(f"All items in '{key}' must be integers")
            
            return self._set_value(key, value)
        
        return property(getter, setter, doc=doc)
    
    @staticmethod
    def _tuple_list_prop(key, doc):
        """Create a property for lists of (X, Y) coordinate tuples."""
        def getter(self):
            return self._get_value(key)
        
        def setter(self, value):
            if value is None:
                return self._del_value(key)
            
            # Validate list of (X, Y) tuples
            if not isinstance(value, list):
                raise TypeError(f"Value for '{key}' must be a list, got {type(value).__name__}")
            
            for i, item in enumerate(value):
                if not isinstance(item, tuple) or len(item) != 2:
                    raise TypeError(f"Item {i} in '{key}' must be a tuple of length 2")
                if not all(isinstance(coord, int) for coord in item):
                    raise TypeError(f"All coordinates in item {i} of '{key}' must be integers")
            
            return self._set_value(key, value)
        
        return property(getter, setter, doc=doc)
    
    @staticmethod
    def _gradation_color_prop(key, doc):
        """Create a property for gradation color lists (2-10 colors)."""
        def getter(self):
            return self._get_value(key)
        
        def setter(self, value):
            if value is None:
                return self._del_value(key)
            
            # Validate list with length constraints
            if not isinstance(value, list):
                raise TypeError(f"Value for '{key}' must be a list, got {type(value).__name__}")
            
            if not (2 <= len(value) <= 10):
                raise ValueError(f"Length of '{key}' must be between 2 and 10, got {len(value)}")
            
            return self._set_value(key, value)
        
        return property(getter, setter, doc=doc)


In [ ]:
_fonttype_map ={0: "don't care", 1: "TTF", 2: "HFT", "dontcare": 0, "ttf": 1, "htf": 2}
reverse_mapping_fonttype_map = {v: k for k, v in _fonttype_map.items()}
reverse_mapping_fonttype_map


In [ ]:
#| export 
_direction_map = {"left": 0, "right": 1, "top": 2, "bottom": 3}
_cap_full_size_map = {"exclude": 0 , "include": 1}
_alignment_map = {"left": 0, "center": 1, "right": 2}
_fonttype_map ={"don't care": 0, "TTF":  1, "HFT": 2, "dontcare": 0, "ttf": 1, "htf": 2}
_shadow_type_map = {"none": 0, "drop": 1, "continous": 2}
_background_type_map = {"empty": 0, "fill": 1, "picture": 2, "gradation":3}
_gradataion_type_map = {"stripe": 1, "circle": 2, "cone": 3, "square": 4}
_rotation_setting_map = {"none": 0, "setted_rotation": 1, "picture_centered_roation": 2, "rotation_and_centered": 3 }
_search_direction_map = {"down": 0, "up": 1, "doc": 2}
_text_direction_map = {"horizontal": 0, "vertical": 1}
_line_wrap_map = {"basic": 0, "no_newline": 1, "forced": 2}
_text_wrap_map = {"square": 0, "top_bottom": 1, "behind": 2, "front": 3, "tight": 4, "through": 5}
_text_flow_map = {"both": 0, "left": 1, "right": 2, "largest": 3}
_vert_align_map = {"top": 0, "center": 1, "bottom": 2}
_vert_rel_to_map = {"paper": 0, "page": 1, "paragraph": 2}
_hroz_rel_to_map = {"paper": 0, "page": 1, "column": 2, "paragraph": 3}
_hroz_align_map = {"left": 0, "center": 1, "right": 2, "inside": 3, "outside": 4}
_align_map = {"left": 0, "center": 1, "right": 2}
_align_type_map = {"between": 0, "left": 1, "right": 2, "center": 3, "ratio": 4, "shared": 5}
_latin_line_break_map = {"word": 0, "hyphen": 1, "letter": 2}
_nonlatin_line_break_map = {"word": 0, "letter": 1}
_text_align_map = {"font": 0, "up": 1, "middle": 2, "donw": 3}
_heading_type_map = {"none": 0, "outline": 1, "number": 2, "bullet": 3}
_border_text_map = {"column": 0, "text": 1}
_underline_type_map = {"none": 0 , "bottom" : 1, "center": 2, "top": 3}
_outline_type_map = {
        "none": 0,
        "solid": 1,
        "dot": 2 ,
        "thick": 3,
        "dash": 4,
        "dashdot": 5,
        "dashdotdot": 6,
    }
_strikeout_type_map = {
        "none": 0,
        "red single": 1,
        "red double": 2,
        "text single": 3,
        "text double": 4,
    }
_use_kerning_map = {"off": 0, "on": 1}

_diac_sym_mark_map = {"none": 0 , "black circle": 1, "empty circle": 2}
_use_font_space_map = {"off":0 , "on": 1}

_numbering_type_map = {"none": 0, "picture": 1, "table": 2, "equation": 3}
_number_format_map = {"1": 0,
                      "circled 1": 1, 
                      "I": 2,
                      "i": 3, 
                      "A": 4, 
                      "a": 5, 
                      "circled A": 6, 
                      "circled a": 7, 
                      "가": 8, 
                      "동그라미 가": 9,
                      "ㄱ": 10,  
                      "동그라미 ㄱ": 11, 
                      "일": 12, 
                      "一": 13, 
                      "동그라미 一" : 14
}
_line_spacing_type_map = {"font": 0, "fixed": 1, "space": 2}
_pic_effect_map = {"none": 0, "bw": 1, "sepia": 2}
_page_break_map = {'none': 0, "cell": 1, "text": 2}

### BorderFill

In [ ]:
# | export
class BorderFill(ParameterSet):
    """
    ### BorderFill

    5) BorderFill : 테두리/배경의 일반 속성

    | Item ID | Type | SubType | Description |
    | --- | --- | --- | --- |
    | BorderTypeLeft | PIT_UI2 |  | 4방향 테두리 종류 : 왼쪽 [선 종류] |
    | BorderTypeRight | PIT_UI2 |  | 4방향 테두리 종류 : 오른쪽 |
    | BorderTypeTop | PIT_UI2 |  | 4방향 테두리 종류 : 위 |
    | BorderTypeBottom | PIT_UI2 |  | 4방향 테두리 종류 : 아래 |
    | BorderWidthLeft | PIT_UI1 |  | 4방향 테두리 두께 : 왼쪽 [선 굵기] |
    | BorderWidthRight | PIT_UI1 |  | 4방향 테두리 두께 : 오른쪽 |
    | BorderWidthTop | PIT_UI1 |  | 4방향 테두리 두께 : 위 |
    | BorderWidthBottom | PIT_UI1 |  | 4방향 테두리 두께 : 아래 |
    | BorderColorLeft | PIT_UI4 |  | 4방향 테두리 색깔 : 왼쪽 (RGB color: 0x00BBGGRR) |
    | BorderColorRight | PIT_UI4 |  | 4방향 테두리 색깔 : 오른쪽 (RGB color: 0x00BBGGRR) |
    | BorderColorTop | PIT_UI4 |  | 4방향 테두리 색깔 : 위 (RGB color: 0x00BBGGRR) |
    | BorderColorBottom | PIT_UI4 |  | 4방향 테두리 색깔 : 아래 (정수값 그대로 사용) |
    | SlashFlag | PIT_UI2 |  | 대각선 플래그 (비트 플래그 조합) |
    | BackSlashFlag | PIT_UI2 |  | 역대각선 플래그 (비트 플래그 조합) |
    | DiagonalType | PIT_UI2 |  | 대각선 유형 (셀 또는 자동 분할 경계선에 사용) |
    | DiagonalWidth | PIT_UI1 |  | 대각선 두께 (셀 또는 자동 분할 경계선에 사용) |
    | DiagonalColor | PIT_UI4 |  | 대각선 색상 (RGB color: 0x00BBGGRR) |
    | BorderFill3D | PIT_UI1 |  | 3차원 효과: 0 = off, 1 = on |
    | Shadow | PIT_UI1 |  | 그림자 효과: 0 = off, 1 = on |
    | FillAttr | PIT_SET | DrawFillAttr | 배경 채우기 속성 |
    | CrookedSlashFlag | PIT_UI2 |  | 꺾인 대각선 플래그 (bit 0, 1) |
    | BreakCellSeparateLine | PIT_UI1 |  | 자동으로 나뉜 표의 경계선 설정: 0 = 기본, 1 = 사용자 설정 |
    | CounterSlashFlag | PIT_UI1 |  | 슬래쉬 대각선 역방향 플래그: 0 = 순방향, 1 = 역방향 |
    | CounterBackSlashFlag | PIT_UI1 |  | 역슬래쉬 대각선 역방향 플래그: 0 = 순방향, 1 = 역방향 |
    | CenterLineFlag | PIT_UI1 |  | 중심선 플래그: 0 = 없음, 1 = 있음 |
    | CrookedSlashFlag1 | PIT_UI1 |  | 슬래쉬 대각선 꺾임 여부 (저바이트) |
    | CrookedSlashFlag2 | PIT_UI1 |  | 역슬래쉬 대각선 꺾임 여부 (고바이트) |

    ※ 대각선의 형태는 다음의 3가지 아이템으로 결정됩니다:
      - SlashFlag (또는 BackSlashFlag)
      - CrookedSlashFlag1 (또는 CrookedSlashFlag2)
      - CounterSlashFlag (또는 CounterBackSlashFlag)
    """

    def __init__(self, parameterset):
        super().__init__(parameterset)
        self.attributes_names = [
            "border_type_left",
            "border_type_right",
            "border_type_top",
            "border_type_bottom",
            "border_width_left",
            "border_width_right",
            "border_width_top",
            "border_width_bottom",
            "border_color_left",
            "border_color_right",
            "border_color_top",
            "border_color_bottom",
            "diagonal_color",
            "slash_flag",
            "backslash_flag",
            "diagonal_type",
            "diagonal_width",
            "crooked_slash_flag",
            "border_fill_3d",
            "shadow",
            "break_cell_separate_line",
            "counter_slash_flag",
            "counter_back_slash_flag",
            "center_line_flag",
            "crooked_slash_flag1",
            "crooked_slash_flag2",
            "fill_attr",
        ]

    # Integer properties
    border_type_left = ParameterSet._int_prop(
        "BorderTypeLeft", "테두리 유형 (왼쪽): 정수 값을 입력하세요."
    )
    border_type_right = ParameterSet._int_prop(
        "BorderTypeRight", "테두리 유형 (오른쪽): 정수 값을 입력하세요."
    )
    border_type_top = ParameterSet._int_prop(
        "BorderTypeTop", "테두리 유형 (위): 정수 값을 입력하세요."
    )
    border_type_bottom = ParameterSet._int_prop(
        "BorderTypeBottom", "테두리 유형 (아래): 정수 값을 입력하세요."
    )

    border_width_left = ParameterSet._int_prop(
        "BorderWidthLeft", "테두리 두께 (왼쪽): 정수 값을 입력하세요."
    )
    border_width_right = ParameterSet._int_prop(
        "BorderWidthRight", "테두리 두께 (오른쪽): 정수 값을 입력하세요."
    )
    border_width_top = ParameterSet._int_prop(
        "BorderWidthTop", "테두리 두께 (위): 정수 값을 입력하세요."
    )
    border_width_bottom = ParameterSet._int_prop(
        "BorderWidthBottom", "테두리 두께 (아래): 정수 값을 입력하세요."
    )

    # Color properties (with conversion)
    border_color_left = ParameterSet._color_prop(
        "BorderCorlorLeft", "테두리 색상 (왼쪽): 정수 값을 입력하세요."
    )
    border_color_right = ParameterSet._color_prop(
        "BorderColorRight", "테두리 색상 (오른쪽): 정수 값을 입력하세요."
    )
    border_color_top = ParameterSet._color_prop(
        "BorderColorTop", "테두리 색상 (위): 정수 값을 입력하세요."
    )
    border_color_bottom = ParameterSet._int_prop(
        "BorderColorBottom", "테두리 색상 (아래): 정수 값을 입력하세요."
    )

    # 대각선 색상 (setter now completes properly)
    diagonal_color = ParameterSet._int_prop(
        "DiagonalColor", "대각선 색상: 정수 값을 입력하세요."
    )

    # Other integer properties
    slash_flag = ParameterSet._int_prop(
        "SlashFlag", "대각선 플래그: 정수 값을 입력하세요."
    )
    backslash_flag = ParameterSet._int_prop(
        "BackSlashFlag", "역대각선 플래그: 정수 값을 입력하세요."
    )
    diagonal_type = ParameterSet._int_prop(
        "DiagonalType", "대각선 유형: 정수 값을 입력하세요."
    )
    diagonal_width = ParameterSet._int_prop(
        "DiagonalWidth", "대각선 두께: 정수 값을 입력하세요."
    )
    crooked_slash_flag = ParameterSet._int_prop(
        "CrookedSlashFlag", "꺾인 대각선 플래그 (bit 0, 1): 정수 값을 입력하세요."
    )

    # Boolean properties
    border_fill_3d = ParameterSet._bool_prop(
        "BorderFill3D", "3차원 효과: 0 = off, 1 = on"
    )
    shadow = ParameterSet._bool_prop("Shadow", "그림자 효과: 0 = off, 1 = on")
    break_cell_separate_line = ParameterSet._bool_prop(
        "BreakCellSeparateLine",
        "자동으로 나뉜 표의 경계선 설정: 0 = 기본, 1 = 사용자 설정",
    )
    counter_slash_flag = ParameterSet._bool_prop(
        "CounterSlashFlag", "슬래쉬 대각선의 역방향 플래그: 0 = 순방향, 1 = 역방향"
    )
    counter_back_slash_flag = ParameterSet._bool_prop(
        "CounterBackSlashFlag",
        "역슬래쉬 대각선의 역방향 플래그: 0 = 순방향, 1 = 역방향",
    )
    center_line_flag = ParameterSet._bool_prop(
        "CenterLineFlag", "중심선 플래그: 0 = 없음, 1 = 있음"
    )
    crooked_slash_flag1 = ParameterSet._bool_prop(
        "CrookedSlashFlag1", "슬래쉬 대각선의 꺾임 여부 (저바이트): 0 = 아니오, 1 = 예"
    )
    crooked_slash_flag2 = ParameterSet._bool_prop(
        "CrookedSlashFlag2",
        "역슬래쉬 대각선의 꺾임 여부 (고바이트): 0 = 아니오, 1 = 예",
    )

    # Typed property for fill attribute
    fill_attr = ParameterSet._typed_prop("FillAttr", "배경 채우기 속성", lambda: DrawFillAttr)

### Caption

In [ ]:
# | export


class Caption(ParameterSet):
    """
    ### Caption

    8) Caption : 캡션 속성

    | Item ID       | Type    | SubType | Description                                                                 |
    | ------------- | ------- | ------- | --------------------------------------------------------------------------- |
    | Side          | PIT_UI1 |         | 방향. 0 = 왼쪽, 1 = 오른쪽, 2 = 위, 3 = 아래                               |
    | Width         | PIT_I   |         | 캡션 폭 (가로 방향이면 높이를 사용함. 단위: HWPUNIT)                      |
    | Gap           | PIT_I   |         | 캡션과의 사이 간격 (단위: HWPUNIT)                                        |
    | CapFullSize   | PIT_UI1 |         | 캡션 폭에 여백을 포함할지 여부 (0 = 포함 안 함, 1 = 포함함)              |
    """

    def __init__(self, parameterset):
        super().__init__(parameterset)
        self.attributes_names = [
            "side",
            "width",
            "gap",
            "cap_full_size",
        ]

    # 'side' must be one of the allowed values
    side = ParameterSet._mapped_prop(
        "Side", _direction_map, doc="방향: 0 = 왼쪽, 1 = 오른쪽, 2 = 위, 3 = 아래"
    )

    # 'width' and 'gap' are stored in HWPUNIT; convert to/from mili
    width = ParameterSet._unit_prop("Width", unit="mili", doc="캡션 폭 (단위: HWPUNIT)")
    gap = ParameterSet._unit_prop("Gap", unit="mili", doc="캡션과의 간격 (단위: HWPUNIT)")

    # Mapping for 'cap_full_size': 0 means 'exclude', 1 means 'include'
    cap_full_size = ParameterSet._mapped_prop(
        "CapFullSize",
        mapping=_cap_full_size_map,
        doc="캡션 폭에 여백 포함 여부 (CapFullSize): 0 = exclude, 1 = include",
    )

### BulletShape

In [ ]:
# | export


class BulletShape(ParameterSet):
    """
    ### BulletShape

    | Item ID        | Type     | SubType        | Description |
    |----------------|----------|----------------|-------------|
    | HasCharShape   | PIT_UI1  |                | 글자 모양 사용 여부: 0 = 기본, 1 = 사용자 정의 |
    | CharShape      | PIT_SET  | CharShape      | 글자 모양 (HasCharShape가 1일 경우 사용) |
    | WidthAdjust    | PIT_I    |                | 글자 크기 보정 값 (HWPUNIT) |
    | TextOffset     | PIT_I    |                | 텍스트 오프셋 (percent or HWPUNIT) |
    | Alignment      | PIT_UI1  |                | 정렬 방식: 0 = 왼쪽, 1 = 가운데, 2 = 오른쪽 |
    | UseInstWidth   | PIT_UI1  |                | 인스턴스 폭 사용 여부 (on/off) |
    | AutoIndent     | PIT_UI1  |                | 자동 들여쓰기 (on/off) |
    | TextOffsetType | PIT_UI1  |                | 텍스트 오프셋 타입: 0 = percent, 1 = HWPUNIT |
    | BulletChar     | PIT_UI2  |                | 글머리 기호 문자 |
    | HasImage       | PIT_UI1  |                | 이미지 글머리 기호 여부: 0 = 없음, 1 = 있음 |
    | BulletImage    | PIT_SET  | DrawImageAttr  | 글머리 기호 이미지 |
    """

    def __init__(self, parameterset):
        super().__init__(parameterset)
        self.attributes_names = [
            "has_char_shape",
            "char_shape",
            "width_adjust",
            "text_offset",
            "alignment",
            "use_inst_width",
            "auto_indent",
            "text_offset_type",
            "bullet_char",
            "has_image",
            "bullet_image",
        ]

    has_char_shape = ParameterSet._bool_prop(
        "HasCharShape", "글자 모양 사용 여부: 0 = 기본, 1 = 사용자 정의"
    )
    char_shape = ParameterSet._typed_prop("CharShape", "글자 모양", lambda: CharShape)
    width_adjust = ParameterSet._int_prop("WidthAdjust", "글자 크기 보정 값 (HWPUNIT)")
    text_offset = ParameterSet._int_prop(
        "TextOffset", "텍스트 오프셋 (percent or HWPUNIT)"
    )
    alignment = ParameterSet._mapped_prop(
        "Alignment",
        _alignment_map,
        doc="정렬 방식: 0 = 왼쪽, 1 = 가운데, 2 = 오른쪽",
    )
    use_inst_width = ParameterSet._bool_prop(
        "UseInstWidth", "인스턴스 폭 사용 여부 (on/off)"
    )
    auto_indent = ParameterSet._bool_prop("AutoIndent", "자동 들여쓰기 (on/off)")
    text_offset_type = ParameterSet._bool_prop(
        "TextOffsetType", "텍스트 오프셋 타입: 0 = percent, 1 = HWPUNIT"
    )
    bullet_char = ParameterSet._int_prop("BulletChar", "글머리 기호 문자")
    has_image = ParameterSet._bool_prop(
        "HasImage", "이미지 글머리 기호 여부: 0 = 없음, 1 = 있음"
    )
    bullet_image = ParameterSet._typed_prop(
        "BulletImage", "글머리 기호 이미지", lambda: DrawImageAttr
    )

### Cell

In [ ]:
# | export


class Cell(ParameterSet):
    """
    ### Cell

    10) Cell : 셀 속성 정의

    | Item ID      | Type     | SubType   | Description                   |
    |--------------|----------|-----------|-------------------------------|
    | HasMargin    | PIT_UI1  |           | 셀 여백 여부 (on / off)       |
    | Protected    | PIT_UI1  |           | 보호 설정 여부: 0 = off, 1 = on|
    | Header       | PIT_UI1  |           | 헤더 여부: 0 = off, 1 = on     |
    | Width        | PIT_I4   |           | 셀의 너비 (HWPUNIT)           |
    | Height       | PIT_I4   |           | 셀의 높이 (HWPUNIT)           |
    | Editable     | PIT_UI1  |           | 편집 가능 여부: 0 = off, 1 = on|
    | Dirty        | PIT_UI1  |           | 변경 여부: 0 = off, 1 = on     |
    | CellCtrlData | PIT_SET  | CtrlData  | 셀 제어 데이터               |
    """

    def __init__(self, parameterset):
        super().__init__(parameterset)
        self.attributes_names = [
            "has_margin",
            "protected",
            "header",
            "width",
            "height",
            "editable",
            "dirty",
            "cell_ctrl_data",
        ]

    has_margin = ParameterSet._bool_prop(
        "HasMargin", "셀 여백 여부 (on / off): 0 또는 1."
    )
    protected = ParameterSet._bool_prop("Protected", "보호 설정 여부: 0 = off, 1 = on.")
    header = ParameterSet._bool_prop("Header", "헤더 여부: 0 = off, 1 = on.")
    width = ParameterSet._unit_prop("Width", "mili", "셀의 너비 (mm).")
    height = ParameterSet._unit_prop("Height", "mili", "셀의 높이 (mm).")
    editable = ParameterSet._bool_prop("Editable", "편집 가능 여부: 0 = off, 1 = on.")
    dirty = ParameterSet._bool_prop("Dirty", "변경 여부: 0 = off, 1 = on.")
    cell_ctrl_data = ParameterSet._typed_prop(
        "CellCtrlData", "셀 제어 데이터", lambda: CtrlData
    )

### Charshape

In [ ]:
# | export


class CharShape(ParameterSet):
    """
        ### CharShape

        13) CharShape : 글자 모양


    | Item ID | Type | SubType | Description |
    | --- | --- | --- | --- |
    | FaceNameHangul | PIT_BSTR |  | 글꼴 이름 (한글) |
    | FaceNameLatin | PIT_BSTR |  | 글꼴 이름 (영문) |
    | FaceNameHanja | PIT_BSTR |  | 글꼴 이름 (한자) |
    | FaceNameJapanese | PIT_BSTR |  | 글꼴 이름 (일본어) |
    | FaceNameOther | PIT_BSTR |  | 글꼴 이름 (기타) |
    | FaceNameSymbol | PIT_BSTR |  | 글꼴 이름 (심벌) |
    | FaceNameUser | PIT_BSTR |  | 글꼴 이름 (사용자) |
    | FontTypeHangul | PIT_UI1 |  | 폰트 종류 (한글) : 0 = don't care, 1 = TTF, 2 = HFT |
    | FontTypeLatin | PIT_UI1 |  | 폰트 종류 (영문) : 0 = don't care, 1 = TTF, 2 = HFT |
    | FontTypeHanja | PIT_UI1 |  | 폰트 종류 (한자) : 0 = don't care, 1 = TTF, 2 = HFT |
    | FontTypeJapanese | PIT_UI1 |  | 폰트 종류 (일본어) : 0 = don't care, 1 = TTF, 2 = HFT |
    | FontTypeOther | PIT_UI1 |  | 폰트 종류 (기타) : 0 = don't care, 1 = TTF, 2 = HFT |
    | FontTypeSymbol | PIT_UI1 |  | 폰트 종류 (심벌) : 0 = don't care, 1 = TTF, 2 = HFT |
    | FontTypeUser | PIT_UI1 |  | 폰트 종류 (사용자) : 0 = don't care, 1 = TTF, 2 = HFT |
    | SizeHangul | PIT_UI1 |  | 각 언어별 크기 비율. (한글) 10% - 250% |
    | SizeLatin | PIT_UI1 |  | 각 언어별 크기 비율. (영문) 10% - 250% |
    | SizeHanja | PIT_UI1 |  | 각 언어별 크기 비율. (한자) 10% - 250% |
    | SizeJapanese | PIT_UI1 |  | 각 언어별 크기 비율. (일본어) 10% - 250% |
    | SizeOther | PIT_UI1 |  | 각 언어별 크기 비율. (기타) 10% - 250% |
    | SizeSymbol | PIT_UI1 |  | 각 언어별 크기 비율. (심벌) 10% - 250% |
    | SizeUser | PIT_UI1 |  | 각 언어별 크기 비율. (사용자) 10% - 250% |
    | RatioHangul | PIT_UI1 |  | 각 언어별 장평 비율. (한글) 50% - 200% |
    | RatioLatin | PIT_UI1 |  | 각 언어별 장평 비율. (영문) 50% - 200% |
    | RatioHanja | PIT_UI1 |  | 각 언어별 장평 비율. (한자) 50% - 200% |
    | RatioJapanese | PIT_UI1 |  | 각 언어별 장평 비율. (일본어) 50% - 200% |
    | RatioOther | PIT_UI1 |  | 각 언어별 장평 비율. (기타) 50% - 200% |
    | RatioSymbol | PIT_UI1 |  | 각 언어별 장평 비율. (심벌) 50% - 200% |
    | RatioUser | PIT_UI1 |  | 각 언어별 장평 비율. (사용자) 50% - 200% |
    | SpacingHangul | PIT_I1 |  | 각 언어별 자간. (한글) -50% - 50% |
    | SpacingLatin | PIT_I1 |  | 각 언어별 자간. (영문) -50% - 50% |
    | SpacingHanja | PIT_I1 |  | 각 언어별 자간. (한자) -50% - 50% |
    | SpacingJapanese | PIT_I1 |  | 각 언어별 자간. (일본어) -50% - 50% |
    | SpacingOther | PIT_I1 |  | 각 언어별 자간. (기타) -50% - 50% |
    | SpacingSymbol | PIT_I1 |  | 각 언어별 자간. (심벌) -50% - 50% |
    | SpacingUser | PIT_I1 |  | 각 언어별 자간. (사용자) -50% - 50% |
    | OffsetHangul | PIT_I1 |  | 각 언어별 오프셋. (한글) -100% - 100% |
    | OffsetLatin | PIT_I1 |  | 각 언어별 오프셋. (영문) -100% - 100% |
    | OffsetHanja | PIT_I1 |  | 각 언어별 오프셋. (한자) -100% - 100% |
    | OffsetJapanese | PIT_I1 |  | 각 언어별 오프셋. (일본어) -100% - 100% |
    | OffsetOther | PIT_I1 |  | 각 언어별 오프셋. (기타) -100% - 100% |
    | OffsetSymbol | PIT_I1 |  | 각 언어별 오프셋. (심벌) -100% - 100% |
    | OffsetUser | PIT_I1 |  | 각 언어별 오프셋. (사용자) -100% - 100% |
    | Bold | PIT_UI1 |  | Bold : 0 = off, 1 = on |
    | Italic | PIT_UI1 |  | Italic : 0 = off, 1 = on |
    | SmallCaps | PIT_UI1 |  | Small Caps : 0 = off, 1 = on |
    | Emboss | PIT_UI1 |  | Emboss : 0 = off, 1 = on |
    | Engrave | PIT_UI1 |  | Engrave : 0 = off, 1 = on |
    | SuperScript | PIT_UI1 |  | Superscript : 0 = off, 1 = on |
    | SubScript | PIT_UI1 |  | Subscript : 0 = off, 1 = on |
    | UnderlineType | PIT_UI1 |  | 밑줄 종류 :  0 = none, 1 = bottom, 2 = center, 3 = top |
    | OutlineType | PIT_UI1 |  | 외곽선 종류 : 0 = none, 1 = solid, 2 = dot, 3 = thick,  4 = dash, 5 = dashdot, 6 = dashdotdot |
    | ShadowType | PIT_UI1 |  | 그림자 종류 : 0 = none, 1 = drop, 2 = continuous |
    | TextColor | PIT_UI4 |  | 글자색. (COLORREF)RGB color를 나타내기 위한 32비트 값 (0x00BBGGRR) |
    | ShadeColor | PIT_UI4 |  | 음영색. (COLORREF)RGB color를 나타내기 위한 32비트 값 (0x00BBGGRR) |
    | UnderlineShape | PIT_UI1 |  | 밑줄 모양 : 선 종류 |
    | UnderlineColor | PIT_UI4 |  | 밑줄 색 (COLORREF)RGB color를 나타내기 위한 32비트 값 (0x00BBGGRR) |
    | ShadowOffsetX | PIT_I1 |  | 그림자 간격 (X 방향) -100% - 100% |
    | ShadowOffsetY | PIT_I1 |  | 그림자 간격 (Y 방향) -100% - 100% |
    | ShadowColor | PIT_UI4 |  | 그림자 색 (COLORREF)RGB color를 나타내기 위한 32비트 값 (0x00BBGGRR) |
    | StrikeOutType | PIT_UI1 |  | 취소선 종류 : 0 = none, 1 = red single, 2 = red double,  3 = text single, 4 = text double |
    | DiacSymMark | PIT_UI1 |  | 강조점 종류 : 0 = none, 1 = 검정 동그라미, 2 = 속 빈 동그라미 |
    | UseFontSpace | PIT_UI1 |  | 글꼴에 어울리는 빈칸 : 0 = off, 1 = on |
    | UseKerning | PIT_UI1 |  | 커닝 : 0 = off, 1 = on |
    | Height | PIT_I4 |  | 글자 크기 (HWPUNIT) |
    | BorderFill | PIT_SET | BorderFill | 테두리/배경 (한글2007에 새로 추가) |
    """

    def __init__(self, parameterset):
        super().__init__(parameterset)
        self.attributes_names = [
            "facename_hangul",
            "facename_latin",
            "facename_hanja",
            "facename_japanese",
            "facename_other",
            "facename_symbol",
            "facename_user",
            "fonttype_hangul",
            "fonttype_latin",
            "fonttype_hanja",
            "fonttype_japanese",
            "fonttype_other",
            "fonttype_symbol",
            "fonttype_user",
            "size_hangul",
            "size_latin",
            "size_hanja",
            "size_japanese",
            "size_other",
            "size_symbol",
            "size_user",
            "ratio_hangul",
            "ratio_latin",
            "ratio_hanja",
            "ratio_japanese",
            "ratio_other",
            "ratio_symbol",
            "ratio_user",
            "spacing_hangul",
            "spacing_latin",
            "spacing_hanja",
            "spacing_japanese",
            "spacing_other",
            "spacing_symbol",
            "spacing_user",
            "offset_hangul",
            "offset_latin",
            "offset_hanja",
            "offset_japanese",
            "offset_other",
            "offset_symbol",
            "offset_user",
            "bold",
            "italic",
            "small_caps",
            "emboss",
            "engrave",
            "superscript",
            "subscript",
            "underline_type",
            "underline_shape",
            "outline_type",
            "shadow_type",
            "text_color",
            "shade_color",
            "underline_color",
            "shadow_color",
            "shadow_offset_x",
            "shadow_offset_y",
            "strikeout_color",
            "strikeout_type",
            "strikeout_shape",
            "diac_sym_mark",
            "use_font_space",
            "use_kerning",
            "height",
            "border_fill",
        ]

    # Facename properties (strings)
    facename_hangul = ParameterSet._str_prop("FaceNameHangul", "글꼴 이름 (한글)")
    facename_latin = ParameterSet._str_prop("FaceNameLatin", "글꼴 이름 (영문)")
    facename_hanja = ParameterSet._str_prop("FaceNameHanja", "글꼴 이름 (한자)")
    facename_japanese = ParameterSet._str_prop("FaceNameJapanese", "글꼴 이름 (일본어)")
    facename_other = ParameterSet._str_prop("FaceNameOther", "글꼴 이름 (기타)")
    facename_symbol = ParameterSet._str_prop("FaceNameSymbol", "글꼴 이름 (심벌)")
    facename_user = ParameterSet._str_prop("FaceNameUser", "글꼴 이름 (사용자)")

    # FontType properties: mapped from numbers to human‐readable strings
    fonttype_hangul = ParameterSet._mapped_prop(
        "FontTypeHangul",
        _fonttype_map,
        "폰트 종류 (한글): 0 = don't care, 1 = TTF, 2 = HFT",
    )
    fonttype_latin = ParameterSet._mapped_prop(
        "FontTypeLatin",
        _fonttype_map,
        "폰트 종류 (영문): 0 = don't care, 1 = TTF, 2 = HFT",
    )
    fonttype_hanja = ParameterSet._mapped_prop(
        "FontTypeHanja",
        _fonttype_map,
        "폰트 종류 (한자): 0 = don't care, 1 = TTF, 2 = HFT",
    )
    fonttype_japanese = ParameterSet._mapped_prop(
        "FontTypeJapanese",
        _fonttype_map,
        "폰트 종류 (일본어): 0 = don't care, 1 = TTF, 2 = HFT",
    )
    fonttype_other = ParameterSet._mapped_prop(
        "FontTypeOther",
        _fonttype_map,
        "폰트 종류 (기타): 0 = don't care, 1 = TTF, 2 = HFT",
    )
    fonttype_symbol = ParameterSet._mapped_prop(
        "FontTypeSymbol",
        _fonttype_map,
        "폰트 종류 (심벌): 0 = don't care, 1 = TTF, 2 = HFT",
    )
    fonttype_user = ParameterSet._mapped_prop(
        "FontTypeUser",
        _fonttype_map,
        "폰트 종류 (사용자): 0 = don't care, 1 = TTF, 2 = HFT",
    )

    # Size properties (10% ~ 250%)
    size_hangul = ParameterSet._int_prop(
        "SizeHangul", "크기 (한글): 10% - 250%", 10, 250
    )
    size_latin = ParameterSet._int_prop("SizeLatin", "크기 (영문): 10% - 250%", 10, 250)
    size_hanja = ParameterSet._int_prop("SizeHanja", "크기 (한자): 10% - 250%", 10, 250)
    size_japanese = ParameterSet._int_prop(
        "SizeJapanese", "크기 (일본어): 10% - 250%", 10, 250
    )
    size_other = ParameterSet._int_prop("SizeOther", "크기 (기타): 10% - 250%", 10, 250)
    size_symbol = ParameterSet._int_prop(
        "SizeSymbol", "크기 (심벌): 10% - 250%", 10, 250
    )
    size_user = ParameterSet._int_prop("SizeUser", "크기 (사용자): 10% - 250%", 10, 250)

    # Ratio properties (50% ~ 200%)
    ratio_hangul = ParameterSet._int_prop(
        "RatioHangul", "장평 (한글): 50% - 200%", 50, 200
    )
    ratio_latin = ParameterSet._int_prop(
        "RatioLatin", "장평 (영문): 50% - 200%", 50, 200
    )
    ratio_hanja = ParameterSet._int_prop(
        "RatioHanja", "장평 (한자): 50% - 200%", 50, 200
    )
    ratio_japanese = ParameterSet._int_prop(
        "RatioJapanese", "장평 (일본어): 50% - 200%", 50, 200
    )
    ratio_other = ParameterSet._int_prop(
        "RatioOther", "장평 (기타): 50% - 200%", 50, 200
    )
    ratio_symbol = ParameterSet._int_prop(
        "RatioSymbol", "장평 (심벌): 50% - 200%", 50, 200
    )
    ratio_user = ParameterSet._int_prop(
        "RatioUser", "장평 (사용자): 50% - 200%", 50, 200
    )

    # Spacing properties (-50% ~ 50%)
    spacing_hangul = ParameterSet._int_prop(
        "SpacingHangul", "자간 (한글): -50% - 50%", -50, 50
    )
    spacing_latin = ParameterSet._int_prop(
        "SpacingLatin", "자간 (영문): -50% - 50%", -50, 50
    )
    spacing_hanja = ParameterSet._int_prop(
        "SpacingHanja", "자간 (한자): -50% - 50%", -50, 50
    )
    spacing_japanese = ParameterSet._int_prop(
        "SpacingJapanese", "자간 (일본어): -50% - 50%", -50, 50
    )
    spacing_other = ParameterSet._int_prop(
        "SpacingOther", "자간 (기타): -50% - 50%", -50, 50
    )
    spacing_symbol = ParameterSet._int_prop(
        "SpacingSymbol", "자간 (심벌): -50% - 50%", -50, 50
    )
    spacing_user = ParameterSet._int_prop(
        "SpacingUser", "자간 (사용자): -50% - 50%", -50, 50
    )

    # Offset properties (-100% ~ 100%)
    offset_hangul = ParameterSet._int_prop(
        "OffsetHangul", "오프셋 (한글): -100% - 100%", -100, 100
    )
    offset_latin = ParameterSet._int_prop(
        "OffsetLatin", "오프셋 (영문): -100% - 100%", -100, 100
    )
    offset_hanja = ParameterSet._int_prop(
        "OffsetHanja", "오프셋 (한자): -100% - 100%", -100, 100
    )
    offset_japanese = ParameterSet._int_prop(
        "OffsetJapanese", "오프셋 (일본어): -100% - 100%", -100, 100
    )
    offset_other = ParameterSet._int_prop(
        "OffsetOther", "오프셋 (기타): -100% - 100%", -100, 100
    )
    offset_symbol = ParameterSet._int_prop(
        "OffsetSymbol", "오프셋 (심벌): -100% - 100%", -100, 100
    )
    offset_user = ParameterSet._int_prop(
        "OffsetUser", "오프셋 (사용자): -100% - 100%", -100, 100
    )

    # Boolean properties
    bold = ParameterSet._bool_prop("Bold", "Bold: 0 = off, 1 = on")
    italic = ParameterSet._bool_prop("Italic", "Italic: 0 = off, 1 = on")
    small_caps = ParameterSet._bool_prop("SmallCaps", "Small Caps: 0 = off, 1 = on")
    emboss = ParameterSet._bool_prop("Emboss", "Emboss: 0 = off, 1 = on")
    engrave = ParameterSet._bool_prop("Engrave", "Engrave: 0 = off, 1 = on")
    superscript = ParameterSet._bool_prop("SuperScript", "Superscript: 0 = off, 1 = on")
    subscript = ParameterSet._bool_prop("SubScript", "Subscript: 0 = off, 1 = on")

    # Underline type (mapped)
    underline_type = ParameterSet._mapped_prop(
        "UnderlineType",
        _underline_type_map,
        "밑줄 종류: 0 = none, 1 = bottom, 2 = center, 3 = top",
    )
    underline_shape = ParameterSet._int_prop("UnderlineShape", "선모양")

    # Outline type (mapped)

    outline_type = ParameterSet._mapped_prop(
        "OutLineType",
        _outline_type_map,
        "외곽선 종류: 0 = none, 1 = solid, 2 = dot, 3 = thick, 4 = dash, 5 = dashdot, 6 = dashdotdot",
    )

    # Shadow type (mapped)
    shadow_type = ParameterSet._mapped_prop(
        "ShadowType",
        _shadow_type_map,
        "그림자 종류: 0 = none, 1 = drop, 2 = continuous",
    )

    # Color properties
    text_color = ParameterSet._color_prop("TextColor", "글자색 (COLORREF)")
    shade_color = ParameterSet._color_prop("ShadeColor", "음영색 (COLORREF)")
    underline_color = ParameterSet._color_prop("UnderlineColor", "밑줄 색상 (COLORREF)")
    shadow_color = ParameterSet._color_prop("ShadowColor", "그림자 색 (COLORREF)")

    # Shadow offset properties (-100% ~ 100%)
    shadow_offset_x = ParameterSet._int_prop(
        "ShadowOffsetX", "그림자 간격 (X 방향): -100% - 100%", -100, 100
    )
    shadow_offset_y = ParameterSet._int_prop(
        "ShadowOffsetY", "그림자 간격 (Y 방향): -100% - 100%", -100, 100
    )

    # Strikeout properties
    strikeout_color = ParameterSet._color_prop("StrikeOutColor", "취소선 색 (COLORREF)")

    strikeout_type = ParameterSet._mapped_prop(
        "StrikeOutType",
        _strikeout_type_map,
        "취소선 종류: 0 = none, 1 = red single, 2 = red double, 3 = text single, 4 = text double",
    )
    strikeout_shape = ParameterSet._int_prop("StrikeOutShape", "선모양")

    # DiacSymMark (mapped)
    diac_sym_mark = ParameterSet._mapped_prop(
        "DiacSymMark",
        _diac_sym_mark_map,
        "강조점 종류: 0 = none, 1 = black circle, 2 = empty circle",
    )

    # UseFontSpace (mapped)
    use_font_space = ParameterSet._mapped_prop(
        "UseFontSpace", _use_font_space_map, "글꼴에 어울리는 빈칸: 0 = off, 1 = on"
    )

    # UseKerning (mapped)
    use_kerning = ParameterSet._mapped_prop(
        "UseKerning", _use_kerning_map, "커닝: 0 = off, 1 = on"
    )

    # Height property
    height = ParameterSet._unit_prop("Height", "pt", "글자 크기 (HWPUNIT)")

    # BorderFill property (explicit)
    border_fill    = ParameterSet._typed_prop("BorderFill", "테두리/채우기 속성", lambda: BorderFill)

    # Aggregated properties for convenience
    @property
    def facename(self):
        return {
            "한글": self.facename_hangul,
            "영어": self.facename_latin,
            "한자": self.facename_hanja,
            "일어": self.facename_japanese,
            "기타": self.facename_other,
            "기호": self.facename_symbol,
            "사용자": self.facename_user,
        }

    @facename.setter
    def facename(self, value):
        keys = [
            "facename_hangul",
            "facename_latin",
            "facename_hanja",
            "facename_japanese",
            "facename_other",
            "facename_symbol",
            "facename_user",
        ]
        if isinstance(value, str):
            for key in keys:
                setattr(self, key, value)
        elif isinstance(value, list):
            for key, val in zip(keys, value):
                setattr(self, key, val)

    @property
    def fonttype(self):
        return {
            "한글": self.fonttype_hangul,
            "영어": self.fonttype_latin,
            "한자": self.fonttype_hanja,
            "일어": self.fonttype_japanese,
            "기타": self.fonttype_other,
            "기호": self.fonttype_symbol,
            "사용자": self.fonttype_user,
        }

    @fonttype.setter
    def fonttype(self, value):
        keys = [
            "fonttype_hangul",
            "fonttype_latin",
            "fonttype_hanja",
            "fonttype_japanese",
            "fonttype_other",
            "fonttype_symbol",
            "fonttype_user",
        ]
        if isinstance(value, int):
            for key in keys:
                setattr(self, key, value)
        elif isinstance(value, list):
            for key, val in zip(keys, value):
                setattr(self, key, val)

    @property
    def size(self):
        return {
            "한글": self.size_hangul,
            "영어": self.size_latin,
            "한자": self.size_hanja,
            "일어": self.size_japanese,
            "기타": self.size_other,
            "기호": self.size_symbol,
            "사용자": self.size_user,
        }

    @size.setter
    def size(self, value):
        keys = [
            "size_hangul",
            "size_latin",
            "size_hanja",
            "size_japanese",
            "size_other",
            "size_symbol",
            "size_user",
        ]
        if isinstance(value, int):
            for key in keys:
                setattr(self, key, value)
        elif isinstance(value, list):
            for key, val in zip(keys, value):
                setattr(self, key, val)

    @property
    def ratio(self):
        return {
            "한글": self.ratio_hangul,
            "영어": self.ratio_latin,
            "한자": self.ratio_hanja,
            "일어": self.ratio_japanese,
            "기타": self.ratio_other,
            "기호": self.ratio_symbol,
            "사용자": self.ratio_user,
        }

    @ratio.setter
    def ratio(self, value):
        keys = [
            "ratio_hangul",
            "ratio_latin",
            "ratio_hanja",
            "ratio_japanese",
            "ratio_other",
            "ratio_symbol",
            "ratio_user",
        ]
        if isinstance(value, int):
            for key in keys:
                setattr(self, key, value)
        elif isinstance(value, list):
            for key, val in zip(keys, value):
                setattr(self, key, val)

    @property
    def spacing(self):
        return {
            "한글": self.spacing_hangul,
            "영어": self.spacing_latin,
            "한자": self.spacing_hanja,
            "일어": self.spacing_japanese,
            "기타": self.spacing_other,
            "기호": self.spacing_symbol,
            "사용자": self.spacing_user,
        }

    @spacing.setter
    def spacing(self, value):
        keys = [
            "spacing_hangul",
            "spacing_latin",
            "spacing_hanja",
            "spacing_japanese",
            "spacing_other",
            "spacing_symbol",
            "spacing_user",
        ]
        if isinstance(value, int):
            for key in keys:
                setattr(self, key, value)
        elif isinstance(value, list):
            for key, val in zip(keys, value):
                setattr(self, key, val)

    @property
    def offset(self):
        return {
            "한글": self.offset_hangul,
            "영어": self.offset_latin,
            "한자": self.offset_hanja,
            "일어": self.offset_japanese,
            "기타": self.offset_other,
            "기호": self.offset_symbol,
            "사용자": self.offset_user,
        }

    @offset.setter
    def offset(self, value):
        keys = [
            "offset_hangul",
            "offset_latin",
            "offset_hanja",
            "offset_japanese",
            "offset_other",
            "offset_symbol",
            "offset_user",
        ]
        if isinstance(value, int):
            for key in keys:
                setattr(self, key, value)
        elif isinstance(value, list):
            for key, val in zip(keys, value):
                setattr(self, key, val)

    def __str__(self):
        attributes = {
            "facename": self.facename, 
            "fonttype": self.fonttype,
            "size": self.size,
            "ratio": self.ratio,
            "spacing": self.spacing,
            "Offset": self.offset,
            "bold": self.bold,
            "italic": self.italic,
            "small_caps": self.small_caps,
            "emboss": self.emboss,
            "engrave": self.engrave,
            "superscript": self.superscript,
            "subscript": self.subscript,
            "underline_type": self.underline_type,
            "outline_type": self.outline_type,
            "shadow_type": self.shadow_type,
            "text_color": self.text_color,
            "shade_color": self.shade_color,
            "underline_colo": self.underline_color,
            "shadow_offset_x": self.shadow_offset_x,
            "shadow_offset_y": self.shadow_offset_y,
            "shadow_color": self.shadow_color,
            "strikeout_type": self.strikeout_type,
            "diac_sym_mark": self.diac_sym_mark,
            "use_font_space": self.use_font_space,
            "use_kerning": self.use_kerning,
            "height": self.height,
            "border_fill": self.border_fill.serialize() if self.border_fill else None,
            }
        data = {"name": self.__class__.__name__, "values": attributes}
        return pprint.pformat(data, indent=4, width=60)

    def __repr__(self):
        return self.__str__()

### CtrlData

In [ ]:
#| export

class CtrlData(ParameterSet):
    """
    ### CtrlData

    22) CtrlData : 제어 데이터

    | Item ID | Type      | SubType | Description          |
    |---------|-----------|---------|----------------------|
    | Name    | PIT_BSTR  |         | 제어 데이터의 이름.  |
    """
    name = ParameterSet._str_prop("Name", "제어 데이터의 이름.")



### DrawArcType

In [ ]:
#| export

class DrawArcType(ParameterSet):
    """
    ### DrawArcType

    27) DrawArcType: 곡선 그리기의 필수적인 속성을 나타내는 클래스

    | Item ID  | Type       | SubType | Description |
    |----------|------------|---------|-------------|
    | Type     | PIT_UI     |         | 곡선 유형: 0 = 선, 1 = 필수곡선, 2 = 화살표 |
    | Interval | PIT_ARRAY  | PIT_I   | 곡선의 시작점과 끝점을 나타내는 배열 |
    """
    _ark_type_map = {"line": 0, "essential": 1, "arrow": 2}
    type = ParameterSet._mapped_prop("Type", _ark_type_map,
                                     doc="곡선 유형: 0 = 선, 1 = 필수곡선, 2 = 화살표")
    interval = ParameterSet._int_list_prop("Interval", "곡선의 시작점과 끝점을 나타내는 배열")



### DrawCoordInfo

In [ ]:
#| export

class DrawCoordInfo(ParameterSet):
    """
    ### DrawCoordInfo

    28) DrawCoordInfo : 그리기 좌표의 상세 정보

    CoordInfo(한글2005)에서 DrawCoordInfo로 이름이 변경되었습니다. 정보를 읽고 쓸 수 있도록 지원합니다.

    | Item ID | Type       | SubType | Description                                 |
    |---------|------------|---------|---------------------------------------------|
    | Count   | PIT_UI4    |         | 점의 개수                                  |
    | Point   | PIT_ARRAY  | PIT_I   | 좌표 Array (X1,Y1), (X2,Y2), ..., (Xn,Yn)   |
    | Line    | PIT_ARRAY  | PIT_UI1 | 선 정보 Array(점들에서 연결된 형태)          |
    """
    count = ParameterSet._int_prop("Count", "점의 개수: 정수 값을 입력하세요.")
    point = ParameterSet._tuple_list_prop("Point", "좌표 배열 (X1, Y1), (X2, Y2), ..., (Xn, Yn): 리스트 값을 입력하세요.")
    line  = ParameterSet._int_list_prop("Line", "선 정보 배열: 리스트 값을 입력하세요.")



### DrawCtrlHyperLink

In [ ]:
#| export

class DrawCtrlHyperlink(ParameterSet):
    """
    ### DrawCtrlHyperlink

    29) DrawCtrlHyperlink : 그림 개체의 Hyperlink 정보

    CtrlHyperlink(한글2005)에서 DrawCtrlHyperlink로 이름이 변경되었습니다.

    | Item ID | Type      | SubType | Description                          |
    |---------|-----------|---------|--------------------------------------|
    | Command | PIT_BSTR  |         | Command String (명령 문자열)         |
    """
    command = ParameterSet._str_prop("Command", "Command String: 명령 문자열을 입력하세요.")



### DrawEditDetail

In [ ]:
#| export

class DrawEditDetail(ParameterSet):
    """
    ### DrawEditDetail

    30) DrawEditDetail : 그림의 교정과 관련된 상세 설정

    | Item ID | Type   | SubType | Description |
    |---------|--------|---------|-------------|
    | Command | PIT_UI |         | Reserved    |
    | Index   | PIT_UI |         | 교점 정의의 인덱스 |
    | PointX  | PIT_I  |         | 교점의 X 좌표 |
    | PointY  | PIT_I  |         | 교점의 Y 좌표 |
    """
    command = ParameterSet._int_prop("Command", "Command: Reserved.")
    index   = ParameterSet._int_prop("Index", "Index: 교점 정의의 인덱스.")
    point_x = ParameterSet._int_prop("PointX", "PointX: 교점의 X 좌표.")
    point_y = ParameterSet._int_prop("PointY", "PointY: 교점의 Y 좌표.")


### DrawFillAttr

In [ ]:
#| export

class DrawFillAttr(ParameterSet):
    """
    ### DrawFillAttr

    31) DrawFillAttr : 그리기 개체의 채우기 속성

    | Item ID               | Type         | SubType   | Description                                                                                         |
    |-----------------------|--------------|-----------|-----------------------------------------------------------------------------------------------------|
    | Type                  | PIT_UI       |           | 배경 유형: 0 = 채우기 없음, 1 = 단색 또는 무늬 채우기, 2 = 그림, 3 = 그라데이션                      |
    | GradationType         | PIT_I        |           | 그라데이션 형태: 1 = 줄무늬형, 2 = 원형, 3 = 원뿔형, 4 = 사각형                                       |
    | GradationAngle        | PIT_I        |           | 그라데이션의 기울기(회전 각도)                                                                       |
    | GradationCenterX      | PIT_I        |           | 그라데이션의 가로 중심 (X 좌표)                                                                      |
    | GradationCenterY      | PIT_I        |           | 그라데이션의 세로 중심 (Y 좌표)                                                                      |
    | GradationStep         | PIT_I        |           | 그라데이션 단계 설정 (0..100)                                                                        |
    | GradationColorNum     | PIT_I        |           | 그라데이션의 색 수                                                                                  |
    | GradationColor        | PIT_ARRAY    | PIT_I     | 그라데이션의 색상 (시작색, 중간색들, 끝색), 길이는 2~10                                               |
    | GradationIndexPos     | PIT_ARRAY    | PIT_I     | 그라데이션 단계 색상들과의 위치                                                                      |
    | GradationStepCenter   | PIT_UI1      |           | 그라데이션 단계 설정의 중심 (0..100)                                                                 |
    | GradationAlpha        | PIT_UI1      |           | 그라데이션 투명도 (0..255)                                                                           |
    | WinBrushFaceColor     | PIT_UI       |           | 단색 (RGB 0x00BBGGRR)                                                                                |
    | WinBrushHatchColor    | PIT_UI       |           | 무늬 (RGB 0x00BBGGRR)                                                                                |
    | WinBrushFaceStyle     | PIT_I1       |           | 무늬 스타일 (0~6)                                                                                   |
    | WinBrushAlpha         | PIT_UI       |           | 단색/무늬 투명도 (0..255)                                                                             |
    | FileName              | PIT_BSTR     |           | 그림 파일 경로                                                                                      |
    | Embedded              | PIT_UI1      |           | 그림이 문서에 직접 삽입(TRUE) / 파일로 연결(FALSE)                                                    |
    | PicEffect             | PIT_UI1      |           | 그림 효과: 0 = 그대로, 1 = 그레이스케일, 2 = 흑백 효과                                               |
    | Brightness            | PIT_I1       |           | 명도 (-100 ~ 100)                                                                                   |
    | Contrast              | PIT_I1       |           | 대비 (-100 ~ 100)                                                                                   |
    | Reverse               | PIT_UI1      |           | 반전 유무 (0 또는 1)                                                                                |
    | DrawFillImageType     | PIT_I        |           | 배경 채우는 방식 (0~14)                                                                              |
    | SkipLeft              | PIT_UI       |           | 왼쪽 자르기                                                                                        |
    | SkipRight             | PIT_UI       |           | 오른쪽 자르기                                                                                      |
    | SkipTop               | PIT_UI       |           | 위 자르기                                                                                          |
    | SkipBottom            | PIT_UI       |           | 아래 자르기                                                                                        |
    | OriginalSizeX         | PIT_I        |           | 이미지 원본 크기 X                                                                                   |
    | OriginalSizeY         | PIT_I        |           | 이미지 원본 크기 Y                                                                                   |
    | InsideMarginLeft      | PIT_I4       |           | 이미지 안쪽 여백 (왼쪽)                                                                              |
    | InsideMarginRight     | PIT_I4       |           | 이미지 안쪽 여백 (오른쪽)                                                                            |
    | InsideMarginTop       | PIT_I4       |           | 이미지 안쪽 여백 (위)                                                                                |
    | InsideMarginBottom    | PIT_I4       |           | 이미지 안쪽 여백 (아래)                                                                              |
    | WindowsBrush          | PIT_UI1      |           | 면/무늬 브러시 여부 (0 또는 1)                                                                       |
    | GradationBrush        | PIT_UI1      |           | 그러데이션 브러시 여부 (0 또는 1)                                                                    |
    | ImageBrush            | PIT_UI1      |           | 그림 브러시 여부 (0 또는 1)                                                                          |
    | ImageCreateOnDrag     | PIT_UI1      |           | 그림 개체 생성 시 드래그 여부 (0 또는 1)                                                             |
    | ImageAlpha            | PIT_UI1      |           | 그림 개체/배경 투명도 (0..255)                                                                        |
    """

    # Basic Properties
    
    type = ParameterSet._mapped_prop("Type", _background_type_map,
                                     doc="배경 유형: 0 = 채우기 없음, 1 = 단색 또는 무늬 채우기, 2 = 그림, 3 = 그라데이션")
    gradation_type = ParameterSet._mapped_prop("GradationType", _gradataion_type_map,
                                             doc="그라데이션 형태: 1 = 줄무늬형, 2 = 원형, 3 = 원뿔형, 4 = 사각형")
    gradation_angle = ParameterSet._int_prop("GradationAngle", "그라데이션의 기울기(회전 각도)")
    gradation_center_x = ParameterSet._int_prop("GradationCenterX", "그라데이션의 가로 중심 (X 좌표)")
    gradation_center_y = ParameterSet._int_prop("GradationCenterY", "그라데이션의 세로 중심 (Y 좌표)")
    gradation_step = ParameterSet._int_prop("GradationStep", "그라데이션 단계 설정 (0..100)", 0, 100)
    gradation_color_num = ParameterSet._int_prop("GradationColorNum", "그라데이션의 색 수")
    
    gradation_color = ParameterSet._gradation_color_prop("GradationColor", "그라데이션의 색상 (시작색, 중간색들, 끝색)")
    
    gradation_index_pos = ParameterSet._int_list_prop("GradationIndexPos", "그라데이션 단계 색상들과의 위치")
    gradation_step_center = ParameterSet._int_prop("GradationStepCenter", "그라데이션 단계 설정의 중심 (0..100)", 0, 100)
    gradation_alpha = ParameterSet._int_prop("GradationAlpha", "그라데이션 투명도", 0, 255)
    
    win_brush_face_color = ParameterSet._int_prop("WinBrushFaceColor", "단색 (RGB 0x00BBGGRR)", 0, 0xFFFFFF)
    win_brush_hatch_color = ParameterSet._int_prop("WinBrushHatchColor", "무늬 (RGB 0x00BBGGRR)", 0, 0xFFFFFF)
    win_brush_face_style = ParameterSet._int_prop("WinBrushFaceStyle", "무늬 스타일", 0, 6)
    win_brush_alpha = ParameterSet._int_prop("WinBrushAlpha", "단색/무늬 투명도", 0, 255)
    
    filename = ParameterSet._str_prop("FileName", "그림 파일 경로")
    embedded = ParameterSet._bool_prop("Embedded", "그림이 문서에 직접 삽입(TRUE) / 파일로 연결(FALSE)")
    _pic_effect_map = {"none": 0, "gray": 1, "black and white": 2}
    pic_effect = ParameterSet._mapped_prop("PicEffect", _pic_effect_map,
                                           doc="그림 효과: 0 = 실제 이미지 그대로, 1 = 그레이스케일, 2 = 흑백 효과")
    brightness = ParameterSet._int_prop("Brightness", "명도 (-100 ~ 100)", -100, 100)
    contrast = ParameterSet._int_prop("Contrast", "대비 (-100 ~ 100)", -100, 100)
    reverse = ParameterSet._bool_prop("Reverse", "반전 유무: 0 (FALSE) 또는 1 (TRUE)")
    draw_fill_image_type = ParameterSet._int_prop("DrawFillImageType", "배경을 채우는 방식", 0, 14)
    
    skip_left = ParameterSet._int_prop("SkipLeft", "왼쪽 자르기")
    skip_right = ParameterSet._int_prop("SkipRight", "오른쪽 자르기")
    skip_top = ParameterSet._int_prop("SkipTop", "위 자르기")
    skip_bottom = ParameterSet._int_prop("SkipBottom", "아래 자르기")
    
    original_size_x = ParameterSet._int_prop("OriginalSizeX", "이미지 원본 크기 X")
    original_size_y = ParameterSet._int_prop("OriginalSizeY", "이미지 원본 크기 Y")
    
    inside_margin_left = ParameterSet._int_prop("InsideMarginLeft", "이미지 안쪽 여백 (왼쪽)")
    inside_margin_right = ParameterSet._int_prop("InsideMarginRight", "이미지 안쪽 여백 (오른쪽)")
    inside_margin_top = ParameterSet._int_prop("InsideMarginTop", "이미지 안쪽 여백 (위)")
    inside_margin_bottom = ParameterSet._int_prop("InsideMarginBottom", "이미지 안쪽 여백 (아래)")
    
    windows_brush = ParameterSet._bool_prop("WindowsBrush", "면/무늬 브러시 여부 (0 또는 1)")
    gradation_brush = ParameterSet._bool_prop("GradationBrush", "그러데이션 브러시 여부 (0 또는 1)")
    image_brush = ParameterSet._bool_prop("ImageBrush", "그림 브러시 여부 (0 또는 1)")
    image_create_on_drag = ParameterSet._bool_prop("ImageCreateOnDrag", "마우스로 그림 개체 생성 여부 (0 또는 1)")
    image_alpha = ParameterSet._int_prop("ImageAlpha", "그림 개체/배경 투명도", 0, 255)


In [ ]:
for level in range(7):
        print(f"""
    @property
    def has_char_shape_level{level}(self):
        return self._get_level_property('HasCharShape', {level})

    @has_char_shape_level{level}.setter
    def has_char_shape_level{level}(self, value):
        self._validate_ui1(value)
        self._set_level_property('HasCharShape', {level}, value)

    @property
    def char_shape_level{level}(self):
        return self._get_level_property('CharShape', {level})

    @char_shape_level{level}.setter
    def char_shape_level{level}(self, value):
        self._set_level_property('CharShape', {level}, value)

    @property
    def width_adjust_level{level}(self):
        return self._get_level_property('WidthAdjust', {level})

    @width_adjust_level{level}.setter
    def width_adjust_level{level}(self, value):
        self._set_level_property('WidthAdjust', {level}, value)

    @property
    def text_offset_level{level}(self):
        return self._get_level_property('TextOffset', {level})

    @text_offset_level{level}.setter
    def text_offset_level{level}(self, value):
        self._set_level_property('TextOffset', {level}, value)

    @property
    def alignment_level{level}(self):
        return self._get_level_property('Alignment', {level})

    @alignment_level{level}.setter
    def alignment_level{level}(self, value):
        self._validate_alignment(value)
        self._set_level_property('Alignment', {level}, value)

    @property
    def use_inst_width_level{level}(self):
        return self._get_level_property('UseInstWidth', {level})

    @use_inst_width_level{level}.setter
    def use_inst_width_level{level}(self, value):
        self._validate_ui1(value)
        self._set_level_property('UseInstWidth', {level}, value)

    @property
    def auto_indent_level{level}(self):
        return self._get_level_property('AutoIndent', {level})

    @auto_indent_level{level}.setter
    def auto_indent_level{level}(self, value):
        self._validate_ui1(value)
        self._set_level_property('AutoIndent', {level}, value)

    @property
    def text_offset_type_level{level}(self):
        return self._get_level_property('TextOffsetType', {level})

    @text_offset_type_level{level}.setter
    def text_offset_type_level{level}(self, value):
        self._validate_ui1(value)
        self._set_level_property('TextOffsetType', {level}, value)

    @property
    def str_format_level{level}(self):
        return self._get_level_property('StrFormat', {level})

    @str_format_level{level}.setter
    def str_format_level{level}(self, value):
        self._set_level_property('StrFormat', {level}, value)

    @property
    def num_format_level{level}(self):
        return self._get_level_property('NumFormat', {level})

    @num_format_level{level}.setter
    def num_format_level{level}(self, value):
        self._validate_ui1(value)
        self._set_level_property('NumFormat', {level}, value)
        """)

### DrawImageAttr

In [ ]:
#| export

class DrawImageAttr(ParameterSet):
    """
    ### DrawImageAttr

    32) DrawImageAttr : 그림 개체 속성

    ImageAttr(한글2005)에서 DrawImageAttr로 이름이 변경되었다.
    그림 개체의 속성을 지정하기 위한 파라메터셋.
    DrawFillAttr에서 그림과 관련된 속성만 빼서 파라메터셋으로 지정되었다.
    (현재 DrawFillAttr와 상속관계가 지정되지 않음)

    | Item ID              | Type     | SubType   | Description                                                                                         |
    |----------------------|----------|-----------|-----------------------------------------------------------------------------------------------------|
    | FileName             | PIT_BSTR |           | ShapeObject의 배경을 그림으로 선택했을 경우 또는 그림개체일 경우의 그림파일 경로                    |
    | Embedded             | PIT_UI1  |           | 그림이 문서에 삽입(T)/링크(F)                                                                         |
    | PicEffect            | PIT_UI1  |           | 그림 효과: 0 = 그대로, 1 = 흑백, 2 = 세피아 등                                                      |
    | Brightness           | PIT_I1   |           | 명도 (-100 ~ 100)                                                                                   |
    | Contrast             | PIT_I1   |           | 밝기 (-100 ~ 100)                                                                                   |
    | Reverse              | PIT_UI1  |           | 반전 유무                                                                                           |
    | DrawFillImageType    | PIT_I    |           | 배경을 채우는 방식 (0~14)                                                                             |
    | SkipLeft             | PIT_UI   |           | 그림 개체일 경우에만 의미 있는 아이템, 왼쪽 자르기                                                  |
    | SkipRight            | PIT_UI   |           | 그림 개체일 경우에만 의미 있는 아이템, 오른쪽 자르기                                                |
    | SkipTop              | PIT_UI   |           | 그림 개체일 경우에만 의미 있는 아이템, 위 자르기                                                    |
    | SkipBottom           | PIT_UI   |           | 그림 개체일 경우에만 의미 있는 아이템, 아래 자르기                                                  |
    | OriginalSizeX        | PIT_UI   |           | 이미지 원본 크기 X                                                                                   |
    | OriginalSizeY        | PIT_UI   |           | 이미지 원본 크기 Y                                                                                   |
    | InsideMarginLeft     | PIT_I4   |           | 이미지 안쪽 여백 (왼쪽)                                                                              |
    | InsideMarginRight    | PIT_I4   |           | 이미지 안쪽 여백 (오른쪽)                                                                            |
    | InsideMarginTop      | PIT_I4   |           | 이미지 안쪽 여백 (위)                                                                                |
    | InsideMarginBottom   | PIT_I4   |           | 이미지 안쪽 여백 (아래)                                                                              |
    | WindowsBrush         | PIT_UI1  |           | 현재 선택된 brush의 type이 면/무늬 브러시 여부                                                       |
    | GradationBrush       | PIT_UI1  |           | 현재 선택된 brush의 type이 그러데이션 브러시 여부                                                    |
    | ImageBrush           | PIT_UI1  |           | 현재 선택된 brush의 type이 그림 브러시 여부                                                          |
    | ImageCreateOnDrag    | PIT_UI1  |           | 그림 개체 생성 시 마우스로 끌어 생성 여부 (한글2007에 새로 추가)                                      |
    | ImageAlpha           | PIT_UI1  |           | 그림 개체/배경 투명도 (0~255)                                                                         |
    """
    file_name = ParameterSet._str_prop("FileName", "그림 파일 경로")
    embedded = ParameterSet._bool_prop("Embedded", "그림이 문서에 삽입(T)/링크(F)")
    pic_effect = ParameterSet._mapped_prop("PicEffect", _pic_effect_map,
                                           doc="그림 효과: 0 = 그대로, 1 = 흑백, 2 = 세피아 등")
    brightness = ParameterSet._int_prop("Brightness", "명도 (-100 ~ 100)", -100, 100)
    contrast = ParameterSet._int_prop("Contrast", "밝기 (-100 ~ 100)", -100, 100)
    reverse = ParameterSet._bool_prop("Reverse", "반전 유무: 0 (off) 또는 1 (on)")
    draw_fill_image_type = ParameterSet._int_prop("DrawFillImageType", "배경을 채우는 방식", 0, 14)
    
    skip_left = ParameterSet._int_prop("SkipLeft", "왼쪽 자르기")
    skip_right = ParameterSet._int_prop("SkipRight", "오른쪽 자르기")
    skip_top = ParameterSet._int_prop("SkipTop", "위 자르기")
    skip_bottom = ParameterSet._int_prop("SkipBottom", "아래 자르기")
    
    original_size_x = ParameterSet._int_prop("OriginalSizeX", "이미지 원본 크기 X")
    original_size_y = ParameterSet._int_prop("OriginalSizeY", "이미지 원본 크기 Y")
    
    inside_margin_left = ParameterSet._int_prop("InsideMarginLeft", "이미지 안쪽 여백 (왼쪽)")
    inside_margin_right = ParameterSet._int_prop("InsideMarginRight", "이미지 안쪽 여백 (오른쪽)")
    inside_margin_top = ParameterSet._int_prop("InsideMarginTop", "이미지 안쪽 여백 (위)")
    inside_margin_bottom = ParameterSet._int_prop("InsideMarginBottom", "이미지 안쪽 여백 (아래)")
    
    windows_brush = ParameterSet._bool_prop("WindowsBrush", "면/무늬 브러시 여부: 0 또는 1")
    gradation_brush = ParameterSet._bool_prop("GradationBrush", "그러데이션 브러시 여부: 0 또는 1")
    image_brush = ParameterSet._bool_prop("ImageBrush", "그림 브러시 여부: 0 또는 1")
    image_create_on_drag = ParameterSet._bool_prop("ImageCreateOnDrag", "그림 개체 생성 시 마우스로 끌어 생성 여부: 0 또는 1")
    image_alpha = ParameterSet._int_prop("ImageAlpha", "그림 개체/배경 투명도 (0~255)", 0, 255)



### DrawImageScissoring

In [ ]:
#| export

class DrawImageScissoring(ParameterSet):
    """
    ### DrawImageScissoring

    33) DrawImageScissoring : 그림 자르기의 좌표 정보

    ImageScissoring(한컴오피스 2005)에서 DrawImageScissoring으로 이름이 변경되었습니다.

    | Item ID       | Type    | SubType | Description          |
    |---------------|---------|---------|----------------------|
    | Xoffset       | PIT_I   |         | 자를 x좌표 오프셋      |
    | Yoffset       | PIT_I   |         | 자를 y좌표 오프셋      |
    | HandleIndex   | PIT_UI  |         | Reserved             |
    """
    x_offset = ParameterSet._int_prop("Xoffset", "자를 x좌표 오프셋: 정수 값을 입력하세요.")
    y_offset = ParameterSet._int_prop("Yoffset", "자를 y좌표 오프셋: 정수 값을 입력하세요.")
    handle_index = ParameterSet._int_prop("HandleIndex", "Reserved: 정수 값을 입력하세요.")


### DrawLayout

In [ ]:
#| export

class DrawLayout(ParameterSet):
    """
    ### DrawLayout

    34) DrawLayout : 도형 레이아웃의 일반 속성

    | Item ID          | Type      | SubType | Description |
    |------------------|-----------|---------|-------------|
    | CreateNumPt      | PIT_UI    |         | 생성할 점의 수 |
    | CreatePt         | PIT_ARRAY | PIT_I   | 생성할 점의 좌표정보: POINT(x,y) 형식의 배열로, CreateNumPt * 2 개수만큼 구성 |
    | CurveSegmentInfo | PIT_ARRAY | PIT_UI1 | 곡선 세그먼트 정보 |
    """
    create_num_pt = ParameterSet._int_prop("CreateNumPt", "생성할 점의 수: 정수 값을 입력하세요.")
    create_pt = ParameterSet._tuple_list_prop("CreatePt", "생성할 점의 좌표정보: (x, y) 튜플의 리스트")
    curve_segment_info = ParameterSet._int_list_prop("CurveSegmentInfo", "곡선 세그먼트 정보: 정수 리스트")


### DrawLineAttr

In [ ]:
#| export 

class DrawLineAttr(ParameterSet):
    """
    ### DrawLineAttr

    35) DrawLineAttr : 도형 선의 속성

    | Item ID        | Type        | SubType | Description                                                 |
    |----------------|-------------|---------|-------------------------------------------------------------|
    | Color          | PMT_UINT32  |         | 선 색상, RGB color를 나타내기 위한 32비트 값 (0x00BBGGRR)    |
    | Style          | PMT_INT     |         | 선의 스타일                                                |
    | Width          | PMT_INT     |         | 선의 두께                                                  |
    | EndCap         | PMT_INT     |         | 선의 끝단                                                  |
    | HeadStyle      | PMT_INT     |         | 선의 시작 화살표 형태                                      |
    | TailStyle      | PMT_INT     |         | 선의 끝 화살표 형태                                        |
    | HeadSize       | PMT_INT     |         | 선의 시작 화살표 크기                                      |
    | TailSize       | PMT_INT     |         | 선의 끝 화살표 크기                                        |
    | HeadFill       | PMT_BOOL    |         | 선의 시작 화살표 채움 여부                                  |
    | TailFill       | PMT_BOOL    |         | 선의 끝 화살표 채움 여부                                    |
    | OutLineStyle   | PMT_UINT    |         | 외곽선 (안쪽/바깥쪽/중앙)                                   |
    | Alpha          | PIT_UI1     |         | 투명도 (한글 2007에 처음 추가됨)                            |
    """
    color         = ParameterSet._int_prop("Color", "선 색상 (RGB 0x00BBGGRR): 정수를 입력하세요.")
    style         = ParameterSet._int_prop("Style", "선의 스타일: 정수를 입력하세요.")
    width         = ParameterSet._int_prop("Width", "선의 두께: 정수를 입력하세요.")
    end_cap       = ParameterSet._int_prop("EndCap", "선의 끝단: 정수를 입력하세요.")
    head_style    = ParameterSet._int_prop("HeadStyle", "선의 시작 화살표 형태: 정수를 입력하세요.")
    tail_style    = ParameterSet._int_prop("TailStyle", "선의 끝 화살표 형태: 정수를 입력하세요.")
    head_size     = ParameterSet._int_prop("HeadSize", "선의 시작 화살표 크기: 정수를 입력하세요.")
    tail_size     = ParameterSet._int_prop("TailSize", "선의 끝 화살표 크기: 정수를 입력하세요.")
    head_fill     = ParameterSet._bool_prop("HeadFill", "선의 시작 화살표 채움 여부: 0 또는 1을 입력하세요.")
    tail_fill     = ParameterSet._bool_prop("TailFill", "선의 끝 화살표 채움 여부: 0 또는 1을 입력하세요.")
    outline_style = ParameterSet._int_prop("OutLineStyle", "외곽선 (안쪽/바깥쪽/중앙): 정수를 입력하세요.")
    alpha         = ParameterSet._int_prop("Alpha", "투명도: 정수를 입력하세요.")


### DrawRectType

In [ ]:
#| export 

class DrawRectType(ParameterSet):
    """
    ### DrawRectType

    36) DrawRectType : 사각형 도형의 속성

    | Item ID | Type    | SubType | Description                           |
    |---------|---------|---------|---------------------------------------|
    | Type    | PIT_UI  |         | 도형의 종류 지정 (0 ~ 50까지)          |
    """
    type = ParameterSet._int_prop("Type", "도형의 종류: 0 ~ 50 사이의 정수 값을 입력하세요.", 0, 50)



### DrawResize

In [ ]:
#| export

class DrawResize(ParameterSet):
    """
    ### DrawResize

    37) DrawResize : 도형 크기 조정 Resizing 정보

    | Item ID      | Type    | SubType | Description                   |
    |--------------|---------|---------|-------------------------------|
    | Xoffset      | PIT_I   |         | 도형 크기 조정 X좌표 오프셋    |
    | Yoffset      | PIT_I   |         | 도형 크기 조정 Y좌표 오프셋    |
    | HandleIndex  | PIT_UI  |         | Reserved                      |
    | Mode         | PIT_UI  |         | Reserved                      |
    """
    x_offset = ParameterSet._int_prop("Xoffset", "도형 크기 조정 X좌표 오프셋: 정수 값을 입력하세요.")
    y_offset = ParameterSet._int_prop("Yoffset", "도형 크기 조정 Y좌표 오프셋: 정수 값을 입력하세요.")
    handle_index = ParameterSet._int_prop("HandleIndex", "Reserved: 정수 값을 입력하세요.")
    mode = ParameterSet._int_prop("Mode", "Reserved: 정수 값을 입력하세요.")


### DrawRotate

In [ ]:
#| export

class DrawRotate(ParameterSet):
    """
    ### DrawRotate

    | Item ID         | Type   | Description                                  |
    |-----------------|--------|----------------------------------------------|
    | Command         | PIT_UI | 회전 설정의 기초 설정 (0: 없음, 1: 설정된 회전, 2: 그림 중심 회전, 3: 회전과 중심) |
    | CenterX         | PIT_I  | 회전 중심의 X 좌표                           |
    | CenterY         | PIT_I  | 회전 중심의 Y 좌표                           |
    | ObjectCenterX   | PIT_I  | 그림 중심의 X 좌표                           |
    | ObjectCenterY   | PIT_I  | 그림 중심의 Y 좌표                           |
    | Angle           | PIT_I  | 회전 각도                                    |
    | RotateImage     | PIT_UI1| 그림 회전 여부 (0: 회전 안 함, 1: 회전함)     |
    """
    command         = ParameterSet._mapped_prop("Command", _rotation_setting_map,
                                                doc="회전 설정의 기초 설정 (0: 없음, 1: 설정된 회전, 2: 그림 중심 회전, 3: 회전과 중심)")
    center_x        = ParameterSet._int_prop("CenterX", "회전 중심의 X 좌표.")
    center_y        = ParameterSet._int_prop("CenterY", "회전 중심의 Y 좌표.")
    object_center_x = ParameterSet._int_prop("ObjectCenterX", "그림 중심의 X 좌표.")
    object_center_y = ParameterSet._int_prop("ObjectCenterY", "그림 중심의 Y 좌표.")
    angle           = ParameterSet._int_prop("Angle", "회전 각도.")
    rotate_image    = ParameterSet._bool_prop("RotateImage", "그림 회전 여부 (0: 회전 안 함, 1: 회전함).")



### DrawScAction

In [ ]:
#| export
class DrawScAction(ParameterSet):
    """
    ### DrawScAction

    39) DrawScAction : 회전 중심과 90도 회전, 좌우/상하 플립 설정

    ScAction(한글2005)에서 DrawScAction으로 이름이 변경되었습니다.

    | Item ID        | Type    | SubType | Description                 |
    |----------------|---------|---------|-----------------------------|
    | RotateCenterX  | PIT_I4  |         | 회전 중심 X 좌표           |
    | RotateCenterY  | PIT_I4  |         | 회전 중심 Y 좌표           |
    | RotateAngel    | PIT_I   |         | 회전각                     |
    | HorzFlip       | PIT_UI  |         | 수평 flip (좌우 대칭 설정) |
    | VertFlip       | PIT_UI  |         | 수직 flip (상하 대칭 설정) |
    """
    rotate_center_x = ParameterSet._int_prop("RotateCenterX", "회전 중심 X 좌표")
    rotate_center_y = ParameterSet._int_prop("RotateCenterY", "회전 중심 Y 좌표")
    rotate_angle    = ParameterSet._int_prop("RotateAngel", "회전각")
    horz_flip       = ParameterSet._bool_prop("HorzFlip", "수평 flip (좌우 대칭 설정)")
    vert_flip       = ParameterSet._bool_prop("VertFlip", "수직 flip (상하 대칭 설정)")



### DrawShadow

In [ ]:
#| export

class DrawShadow(ParameterSet):
    """
    ### DrawShadow

    40) DrawShadow : 그림자 효과 정보

    | Item ID       | Type     | SubType | Description                                     |
    |---------------|----------|---------|-------------------------------------------------|
    | ShadowType    | PIT_I4   |         | 그림자 종류: 0 = none, 1 = drop, 2 = continuous   |
    | ShadowColor   | PIT_UI4  |         | 그림자 색상 (COLORREF)                           |
    | ShadowOffsetX | PIT_I4   |         | 그림자 X축 오프셋 (-48% ~ 48%)                   |
    | ShadowOffsetY | PIT_I4   |         | 그림자 Y축 오프셋 (-48% ~ 48%)                   |
    | ShadowAlpha   | PIT_UI1  |         | 그림자 투명도 (0 ~ 255)                          |
    """
    shadow_type    = ParameterSet._mapped_prop("ShadowType", _shadow_type_map,
                                              doc="그림자 종류: 0 = none, 1 = drop, 2 = continuous")
    shadow_color   = ParameterSet._int_prop("ShadowColor", "그림자 색상 (COLORREF): 정수를 입력하세요.")
    shadow_offset_x = ParameterSet._int_prop("ShadowOffsetX", "그림자 X축 오프셋 (-48% ~ 48%)", -48, 48)
    shadow_offset_y = ParameterSet._int_prop("ShadowOffsetY", "그림자 Y축 오프셋 (-48% ~ 48%)", -48, 48)
    shadow_alpha   = ParameterSet._int_prop("ShadowAlpha", "그림자 투명도 (0 ~ 255)", 0, 255)


### DrawShear

In [ ]:
#| export

class DrawShear(ParameterSet):
    """
    ### DrawShear

    41) DrawShear : Shear transformation parameters

    | Item ID | Type   | SubType | Description        |
    |---------|--------|---------|--------------------|
    | XFactor | PIT_I  |         | X shear factor     |
    | YFactor | PIT_I  |         | Y shear factor     |
    """
    x_factor = ParameterSet._int_prop("XFactor", "X shear factor: 정수 값을 입력하세요.")
    y_factor = ParameterSet._int_prop("YFactor", "Y shear factor: 정수 값을 입력하세요.")



### DrawTextart

In [ ]:
#| export

class DrawTextart(ParameterSet):
    """
    ### DrawTextart

    42) DrawTextart : 텍스트아트 속성
    """
    string         = ParameterSet._str_prop("String", "텍스트아트 내용: 문자열 값을 입력하세요.")
    font_name      = ParameterSet._str_prop("FontName", "폰트 이름.")
    font_style     = ParameterSet._str_prop("FontStyle", "폰트 스타일 (0 = Regular).")
    font_type      = ParameterSet._mapped_prop("FontType", _fonttype_map,
                                             doc="폰트 타입: 0 = don't care, 1 = TTF, 2 = HFT.")
    line_spacing   = ParameterSet._int_prop("LineSpacing", "줄 간격 (50 ~ 500).", 50, 500)
    char_spacing   = ParameterSet._int_prop("CharSpacing", "문자 간격 (50 ~ 500).", 50, 500)
    align_type     = ParameterSet._int_prop("AlignType", "정렬 유형.")
    shape          = ParameterSet._int_prop("Shape", "형태 (0 ~ 54).", 0, 54)
    shadow_type    = ParameterSet._mapped_prop("ShadowType", _shadow_type_map,
                                              doc="그림자 유형: 0 = none, 1 = drop, 2 = continuous.")
    shadow_offset_x = ParameterSet._int_prop("ShadowOffsetX", "그림자 X 오프셋 (-48 ~ 48).", -48, 48)
    shadow_offset_y = ParameterSet._int_prop("ShadowOffsetY", "그림자 Y 오프셋 (-48 ~ 48).", -48, 48)
    shadow_color   = ParameterSet._int_prop("ShadowColor", "그림자 색상 (RGB 32비트).")
    number_of_lines = ParameterSet._int_prop("NumberOfLines", "텍스트아트의 줄 수.")



### FindReplace

In [ ]:
# | export


class FindReplace(ParameterSet):
    """
    ### FindReplace

    56) FindReplace : 찾기/찾아 바꾸기

    | Item ID           | Type      | SubType | Description                                                               |
    |-------------------|-----------|---------|---------------------------------------------------------------------------|
    | FindString        | PIT_BSTR  |         | 찾을 문자열                                                              |
    | ReplaceString     | PIT_BSTR  |         | 바꿀 문자열                                                              |
    | Direction         | PIT_UI1   |         | 찾을 방향 : 0 = 아래쪽, 1 = 위쪽, 2 = 문서 전체                           |
    | MatchCase         | PIT_UI1   |         | 대소문자 구별 (on/off)                                                     |
    | AllWordForms      | PIT_UI1   |         | 모든 단어 형태 (on/off)                                                   |
    | SeveralWords      | PIT_UI1   |         | 여러 단어 찾기 (on/off)                                                    |
    | UseWildCards      | PIT_UI1   |         | 와일드카드 사용 (on/off)                                                   |
    | WholeWordOnly     | PIT_UI1   |         | 전체 단어만 찾기 (on/off)                                                   |
    | AutoSpell         | PIT_UI1   |         | 자동 맞춤법 사용 (on/off)                                                  |
    | ReplaceMode       | PIT_UI1   |         | 찾아 바꾸기 모드 (on/off)                                                  |
    | IgnoreFindString  | PIT_UI1   |         | 찾을 문자열 무시 (on/off)                                                  |
    | IgnoreReplaceString| PIT_UI1  |         | 바꿀 문자열 무시 (on/off)                                                   |
    | FindCharShape     | PIT_SET   | CharShape | 찾을 글자 모양                                                          |
    | FindParaShape     | PIT_SET   | ParaShape | 찾을 문단 모양                                                          |
    | ReplaceCharShape  | PIT_SET   | CharShape | 바꿀 글자 모양                                                          |
    | ReplaceParaShape  | PIT_SET   | ParaShape | 바꿀 문단 모양                                                          |
    | FindStyle         | PIT_BSTR  |         | 찾을 스타일                                                              |
    | ReplaceStyle      | PIT_BSTR  |         | 바꿀 스타일                                                              |
    | IgnoreMessage     | PIT_UI1   |         | 메시지박스 표시 안함 (on/off)                                              |
    | HanjaFromHangul   | PIT_UI1   |         | 한글임으로 한자 차기                                                      |
    | FindJaso          | PIT_UI1   |         | 자소로 찾기 (on/off)                                                      |
    | FindRegExp        | PIT_UI1   |         | 정규식(조건식)으로 찾기 (on/off)                                           |
    | FindType          | PIT_UI1   |         | 찾기 유형 (on/off)                                                       |
    """

    def __init__(self, parameterset):
        super().__init__(parameterset)
        self.attributes_names = [
            "find_string",
            "replace_string",
            "find_style",
            "replace_style",
            "direction",
            "match_case",
            "all_word_forms",
            "several_words",
            "use_wildcards",
            "whole_word_only",
            "auto_spell",
            "replace_mode",
            "ignore_find_string",
            "ignore_replace_string",
            "ignore_message",
            "hanja_from_hangul",
            "find_jaso",
            "find_regexp",
            "find_type",
            "find_charshape",
            "find_parashape",
            "replace_charshape",
            "replace_parashape",
        ]

    # String properties
    find_string = ParameterSet._str_prop("FindString", "찾을 문자열")
    replace_string = ParameterSet._str_prop("ReplaceString", "바꿀 문자열")
    find_style = ParameterSet._str_prop("FindStyle", "찾을 스타일")
    replace_style = ParameterSet._str_prop("ReplaceStyle", "바꿀 스타일")

    # Enum property for direction
    direction = ParameterSet._mapped_prop(
        "Direction",
        _search_direction_map,
        doc="찾을 방향: 0 = 아래쪽, 1 = 위쪽, 2 = 문서 전체",
    )

    # Boolean properties (on/off flags)
    match_case = ParameterSet._bool_prop("MatchCase", "대소문자 구별 (on/off)")
    all_word_forms = ParameterSet._bool_prop("AllWordForms", "모든 단어 형태 (on/off)")
    several_words = ParameterSet._bool_prop("SeveralWords", "여러 단어 찾기 (on/off)")
    use_wildcards = ParameterSet._bool_prop("UseWildCards", "와일드카드 사용 (on/off)")
    whole_word_only = ParameterSet._bool_prop(
        "WholeWordOnly", "전체 단어만 찾기 (on/off)"
    )
    auto_spell = ParameterSet._bool_prop("AutoSpell", "자동 맞춤법 사용 (on/off)")
    replace_mode = ParameterSet._bool_prop("ReplaceMode", "찾아 바꾸기 모드 (on/off)")
    ignore_find_string = ParameterSet._bool_prop(
        "IgnoreFindString", "찾을 문자열 무시 (on/off)"
    )
    ignore_replace_string = ParameterSet._bool_prop(
        "IgnoreReplaceString", "바꿀 문자열 무시 (on/off)"
    )
    ignore_message = ParameterSet._bool_prop(
        "IgnoreMessage", "메시지박스 표시 안함 (on/off)"
    )
    hanja_from_hangul = ParameterSet._bool_prop(
        "HanjaFromHangul", "한글임으로 한자 차기"
    )
    find_jaso = ParameterSet._bool_prop("FindJaso", "자소로 찾기 (on/off)")
    find_regexp = ParameterSet._bool_prop("FindRegExp", "정규식 찾기 (on/off)")
    find_type = ParameterSet._bool_prop("FindType", "찾기 유형 (on/off)")

    # Composite properties using _typed_prop
    find_charshape = ParameterSet._typed_prop(
        "FindCharShape", "찾을 글자 모양", lambda: CharShape
    )
    find_parashape = ParameterSet._typed_prop(
        "FindParaShape", "찾을 문단 모양", lambda: ParaShape
    )
    replace_charshape = ParameterSet._typed_prop(
        "ReplaceCharShape", "바꿀 글자 모양", lambda: CharShape
    )
    replace_parashape = ParameterSet._typed_prop(
        "ReplaceParaShape", "바꿀 문단 모양", lambda: ParaShape
    )

### ListProperties

In [ ]:
#| export

class ListProperties(ParameterSet):
    """
    ### ListProperties

    76) ListProperties : 셀 리스트의 속성

    | Item ID       | Type     | SubType | Description                                      |
    |---------------|----------|---------|--------------------------------------------------|
    | TextDirection | PIT_UI1  |         | 글자 방향 (세로 쓰기 여부를 미정)                |
    | LineWrap      | PIT_UI1  |         | 강제에서 줄 바꿈 0 = 줄바꿈 없는 기본값           |
    | VertAlign     | PIT_UI1  |         | 세로 정렬 0 = 위 정렬                             |
    | MarginLeft    | PIT_I4   |         | 왼쪽 여백                                       |
    | MarginRight   | PIT_I4   |         | 오른쪽 여백                                     |
    | MarginTop     | PIT_I4   |         | 위쪽 여백                                       |
    | MarginBottom  | PIT_I4   |         | 아래쪽 여백                                     |
    """
    text_direction = ParameterSet._mapped_prop("TextDirection", _text_direction_map,
                                               doc="글자 방향: 0 = 기본값, 1 = 세로 쓰기")
    line_wrap = ParameterSet._mapped_prop("LineWrap", _line_wrap_map,
                                          doc="줄 바꿈 옵션: 0 = 기본값, 1 = 줄 바꿈 없음, 2 = 강제 줄 바꿈")
    vert_align = ParameterSet._mapped_prop("VertAlign", _vert_align_map,
                                           doc="세로 정렬: 0 = 위, 1 = 가운데, 2 = 아래")
    margin_left = ParameterSet._int_prop("MarginLeft", "왼쪽 여백: 정수 값을 입력하세요.")
    margin_right = ParameterSet._int_prop("MarginRight", "오른쪽 여백: 정수 값을 입력하세요.")
    margin_top = ParameterSet._int_prop("MarginTop", "위쪽 여백: 정수 값을 입력하세요.")
    margin_bottom = ParameterSet._int_prop("MarginBottom", "아래쪽 여백: 정수 값을 입력하세요.")



### NumberingShape

In [ ]:
#| export

class NumberingShape(ParameterSet):
    """
    ### NumberingShape

    | Item ID                                  | Type      | SubType   | Description                                           |
    |------------------------------------------|-----------|-----------|-------------------------------------------------------|
    | HasCharShapeLevel0 ~ HasCharShapeLevel6   | PIT_UI1   |           | 7단 수준별 지정 여부 (0 = 기본값, 1 = 지정)           |
    | CharShapeLevel0 ~ CharShapeLevel6         | PIT_SET   | CharShape | 수준별 글자 모양 지정                                 |
    | WidthAdjustLevel0 ~ WidthAdjustLevel6     | PIT_I     |           | 수준별 너비 조정 값 (HWPUNIT)                         |
    | TextOffsetLevel0 ~ TextOffsetLevel6       | PIT_I     |           | 수준별 텍스트 오프셋 (percent or HWPUNIT)              |
    | AlignmentLevel0 ~ AlignmentLevel6         | PIT_UI1   |           | 수준별 정렬 (0 = 왼쪽, 1 = 가운데, 2 = 오른쪽)         |
    | UseInstWidthLevel0 ~ UseInstWidthLevel6     | PIT_UI1   |           | 수준별 너비를 문서 내부 너비에 맞출지 여부             |
    | AutoIndentLevel0 ~ AutoIndentLevel6       | PIT_UI1   |           | 수준별 자동 들여쓰기 여부                             |
    | TextOffsetTypeLevel0 ~ TextOffsetTypeLevel6 | PIT_UI1   |           | 텍스트 오프셋 타입 (0 = 기본값, 1 = HWPUNIT)            |
    | StrFormatLevel0 ~ StrFormatLevel6         | PIT_BSTR  |           | 수준별 문자열 포맷                                   |
    | NumFormatLevel0 ~ NumFormatLevel6         | PIT_UI1   |           | 수준별 번호 포맷 (0 또는 1)                            |
    | StartNumber                              | PIT_UI2   |           | 시작 번호 (0 = 기본값, n = 지정 번호)                 |
    | NewList                                  | PIT_UI1   |           | 새 목록 생성 여부                                   |
    """
    start_number = ParameterSet._int_prop("StartNumber", "시작 번호 (0 = 기본값, n = 지정 번호)")
    new_list     = ParameterSet._bool_prop("NewList", "새 목록 생성 여부 (0 = off, 1 = on)")

    # Level 0 properties
    has_char_shape_level0  = ParameterSet._bool_prop("HasCharShapeLevel0", "수준0 지정 여부 (0 = 기본값, 1 = 지정)")
    char_shape_level0      = ParameterSet._typed_prop("CharShapeLevel0", "수준0 글자 모양 지정", lambda: CharShape)
    width_adjust_level0    = ParameterSet._int_prop("WidthAdjustLevel0", "수준0 너비 조정 값 (HWPUNIT)")
    text_offset_level0     = ParameterSet._int_prop("TextOffsetLevel0", "수준0 텍스트 오프셋 (percent or HWPUNIT)")
    alignment_level0       = ParameterSet._mapped_prop("AlignmentLevel0", _align_map,
                                                     doc="수준0 정렬: 0 = 왼쪽, 1 = 가운데, 2 = 오른쪽")
    use_inst_width_level0  = ParameterSet._bool_prop("UseInstWidthLevel0", "수준0 문서 내부 너비 맞춤 여부")
    auto_indent_level0     = ParameterSet._bool_prop("AutoIndentLevel0", "수준0 자동 들여쓰기 여부")
    text_offset_type_level0 = ParameterSet._bool_prop("TextOffsetTypeLevel0", "수준0 텍스트 오프셋 타입 (0 = 기본값, 1 = HWPUNIT)")
    str_format_level0      = ParameterSet._str_prop("StrFormatLevel0", "수준0 문자열 포맷")
    num_format_level0      = ParameterSet._mapped_prop("NumFormatLevel0", _number_format_map,
                                                     doc="수준0 번호 포맷 (0 또는 1)")

    # Level 1 properties
    has_char_shape_level1  = ParameterSet._bool_prop("HasCharShapeLevel1", "수준1 지정 여부")
    char_shape_level1      = ParameterSet._typed_prop("CharShapeLevel1", "수준1 글자 모양 지정", lambda: CharShape)
    width_adjust_level1    = ParameterSet._int_prop("WidthAdjustLevel1", "수준1 너비 조정 값 (HWPUNIT)")
    text_offset_level1     = ParameterSet._int_prop("TextOffsetLevel1", "수준1 텍스트 오프셋")
    alignment_level1       = ParameterSet._mapped_prop("AlignmentLevel1", _align_map,
                                                     doc="수준1 정렬: 0 = 왼쪽, 1 = 가운데, 2 = 오른쪽")
    use_inst_width_level1  = ParameterSet._bool_prop("UseInstWidthLevel1", "수준1 문서 내부 너비 맞춤 여부")
    auto_indent_level1     = ParameterSet._bool_prop("AutoIndentLevel1", "수준1 자동 들여쓰기 여부")
    text_offset_type_level1 = ParameterSet._bool_prop("TextOffsetTypeLevel1", "수준1 텍스트 오프셋 타입")
    str_format_level1      = ParameterSet._str_prop("StrFormatLevel1", "수준1 문자열 포맷")
    num_format_level1      = ParameterSet._mapped_prop("NumFormatLevel1", _number_format_map,
                                                     doc="수준1 번호 포맷")

    # Level 2 properties
    has_char_shape_level2  = ParameterSet._bool_prop("HasCharShapeLevel2", "수준2 지정 여부")
    char_shape_level2      = ParameterSet._typed_prop("CharShapeLevel2", "수준2 글자 모양 지정", lambda: CharShape)
    width_adjust_level2    = ParameterSet._int_prop("WidthAdjustLevel2", "수준2 너비 조정 값 (HWPUNIT)")
    text_offset_level2     = ParameterSet._int_prop("TextOffsetLevel2", "수준2 텍스트 오프셋")
    alignment_level2       = ParameterSet._mapped_prop("AlignmentLevel2", _align_map,
                                                     doc="수준2 정렬: 0 = 왼쪽, 1 = 가운데, 2 = 오른쪽")
    use_inst_width_level2  = ParameterSet._bool_prop("UseInstWidthLevel2", "수준2 문서 내부 너비 맞춤 여부")
    auto_indent_level2     = ParameterSet._bool_prop("AutoIndentLevel2", "수준2 자동 들여쓰기 여부")
    text_offset_type_level2 = ParameterSet._bool_prop("TextOffsetTypeLevel2", "수준2 텍스트 오프셋 타입")
    str_format_level2      = ParameterSet._str_prop("StrFormatLevel2", "수준2 문자열 포맷")
    num_format_level2      = ParameterSet._mapped_prop("NumFormatLevel2", _number_format_map,
                                                     doc="수준2 번호 포맷")

    # Level 3 properties
    has_char_shape_level3  = ParameterSet._bool_prop("HasCharShapeLevel3", "수준3 지정 여부")
    char_shape_level3      = ParameterSet._typed_prop("CharShapeLevel3", "수준3 글자 모양 지정", lambda: CharShape)
    width_adjust_level3    = ParameterSet._int_prop("WidthAdjustLevel3", "수준3 너비 조정 값 (HWPUNIT)")
    text_offset_level3     = ParameterSet._int_prop("TextOffsetLevel3", "수준3 텍스트 오프셋")
    alignment_level3       = ParameterSet._mapped_prop("AlignmentLevel3", _align_map,
                                                     doc="수준3 정렬: 0 = 왼쪽, 1 = 가운데, 2 = 오른쪽")
    use_inst_width_level3  = ParameterSet._bool_prop("UseInstWidthLevel3", "수준3 문서 내부 너비 맞춤 여부")
    auto_indent_level3     = ParameterSet._bool_prop("AutoIndentLevel3", "수준3 자동 들여쓰기 여부")
    text_offset_type_level3 = ParameterSet._bool_prop("TextOffsetTypeLevel3", "수준3 텍스트 오프셋 타입")
    str_format_level3      = ParameterSet._str_prop("StrFormatLevel3", "수준3 문자열 포맷")
    num_format_level3      = ParameterSet._mapped_prop("NumFormatLevel3", _number_format_map,
                                                     doc="수준3 번호 포맷")

    # Level 4 properties
    has_char_shape_level4  = ParameterSet._bool_prop("HasCharShapeLevel4", "수준4 지정 여부")
    char_shape_level4      = ParameterSet._typed_prop("CharShapeLevel4", "수준4 글자 모양 지정", lambda: CharShape)
    width_adjust_level4    = ParameterSet._int_prop("WidthAdjustLevel4", "수준4 너비 조정 값 (HWPUNIT)")
    text_offset_level4     = ParameterSet._int_prop("TextOffsetLevel4", "수준4 텍스트 오프셋")
    alignment_level4       = ParameterSet._mapped_prop("AlignmentLevel4", _align_map,
                                                     doc="수준4 정렬: 0 = 왼쪽, 1 = 가운데, 2 = 오른쪽")
    use_inst_width_level4  = ParameterSet._bool_prop("UseInstWidthLevel4", "수준4 문서 내부 너비 맞춤 여부")
    auto_indent_level4     = ParameterSet._bool_prop("AutoIndentLevel4", "수준4 자동 들여쓰기 여부")
    text_offset_type_level4 = ParameterSet._bool_prop("TextOffsetTypeLevel4", "수준4 텍스트 오프셋 타입")
    str_format_level4      = ParameterSet._str_prop("StrFormatLevel4", "수준4 문자열 포맷")
    num_format_level4      = ParameterSet._mapped_prop("NumFormatLevel4", _number_format_map,
                                                     doc="수준4 번호 포맷")

    # Level 5 properties
    has_char_shape_level5  = ParameterSet._bool_prop("HasCharShapeLevel5", "수준5 지정 여부")
    char_shape_level5      = ParameterSet._typed_prop("CharShapeLevel5", "수준5 글자 모양 지정", lambda: CharShape)
    width_adjust_level5    = ParameterSet._int_prop("WidthAdjustLevel5", "수준5 너비 조정 값 (HWPUNIT)")
    text_offset_level5     = ParameterSet._int_prop("TextOffsetLevel5", "수준5 텍스트 오프셋")
    alignment_level5       = ParameterSet._mapped_prop("AlignmentLevel5", _align_map,
                                                     doc="수준5 정렬: 0 = 왼쪽, 1 = 가운데, 2 = 오른쪽")
    use_inst_width_level5  = ParameterSet._bool_prop("UseInstWidthLevel5", "수준5 문서 내부 너비 맞춤 여부")
    auto_indent_level5     = ParameterSet._bool_prop("AutoIndentLevel5", "수준5 자동 들여쓰기 여부")
    text_offset_type_level5 = ParameterSet._bool_prop("TextOffsetTypeLevel5", "수준5 텍스트 오프셋 타입")
    str_format_level5      = ParameterSet._str_prop("StrFormatLevel5", "수준5 문자열 포맷")
    num_format_level5      = ParameterSet._mapped_prop("NumFormatLevel5", _number_format_map,
                                                     doc="수준5 번호 포맷")

    # Level 6 properties
    has_char_shape_level6  = ParameterSet._bool_prop("HasCharShapeLevel6", "수준6 지정 여부")
    char_shape_level6      = ParameterSet._typed_prop("CharShapeLevel6", "수준6 글자 모양 지정", lambda: CharShape)
    width_adjust_level6    = ParameterSet._int_prop("WidthAdjustLevel6", "수준6 너비 조정 값 (HWPUNIT)")
    text_offset_level6     = ParameterSet._int_prop("TextOffsetLevel6", "수준6 텍스트 오프셋")
    alignment_level6       = ParameterSet._mapped_prop("AlignmentLevel6", _align_map,
                                                     doc="수준6 정렬: 0 = 왼쪽, 1 = 가운데, 2 = 오른쪽")
    use_inst_width_level6  = ParameterSet._bool_prop("UseInstWidthLevel6", "수준6 문서 내부 너비 맞춤 여부")
    auto_indent_level6     = ParameterSet._bool_prop("AutoIndentLevel6", "수준6 자동 들여쓰기 여부")
    text_offset_type_level6 = ParameterSet._bool_prop("TextOffsetTypeLevel6", "수준6 텍스트 오프셋 타입")
    str_format_level6      = ParameterSet._str_prop("StrFormatLevel6", "수준6 문자열 포맷")
    num_format_level6      = ParameterSet._mapped_prop("NumFormatLevel6", _number_format_map,
                                                     doc="수준6 번호 포맷")

        



### ParaShape

In [ ]:
# | export
class ParaShape(ParameterSet):
    """
    ### ParaShape

    91) ParaShape : 문단 모양

    | Item ID          | Type    | SubType | Description |
    |------------------|---------|---------|-------------|
    | LeftMargin       | PIT_I4  |         | 왼쪽 여백 (URC) |
    | RightMargin      | PIT_I4  |         | 오른쪽 여백 (URC) |
    | Indentation      | PIT_I4  |         | 들여쓰기/내어 쓰기 (URC) |
    | PrevSpacing      | PIT_I4  |         | 문단 간격 위 (URC) |
    | NextSpacing      | PIT_I4  |         | 문단 간격 아래 (URC) |
    | LineSpacingType  | PIT_UI1 |         | 줄 간격 종류 (HWPUNIT): 0 = 글꼴 기준, 1 = 고정 값, 2 = 여백만 지정 |
    | LineSpacing      | PIT_I4  |         | 줄 간격 값 |
    | AlignType        | PIT_UI1 |         | 정렬 방식: 0 = 양쪽 정렬, 1 = 왼쪽 정렬, 2 = 오른쪽 정렬, 3 = 가운데 정렬, 4 = 배분 정렬, 5 = 나눔 정렬 |
    | BreakLatinWord   | PIT_UI1 |         | 줄 나눔 방식 (라틴): 0 = 단어, 1 = 하이픈, 2 = 글자 |
    | BreakNonLatinWord| PIT_UI1 |         | 줄 나눔 (비 라틴): 0 = 어절, 1 = 글자 |
    | SnapToGrid       | PIT_UI1 |         | 편집 용지의 줄 격자 사용 (on/off) |
    | Condense         | PIT_UI1 |         | 공백 최소값 (0 - 75%) |
    | WidowOrphan      | PIT_UI1 |         | 외톨이줄 보호 (on/off) |
    | KeepWithNext     | PIT_UI1 |         | 다음 문단과 함께 (on/off) |
    | KeepLinesTogether| PIT_UI1 |         | 문단 보호 (on/off) |
    | PagebreakBefore  | PIT_UI1 |         | 문단 앞에서 항상 쪽 나눔 (on/off) |
    | TextAlignment    | PIT_UI1 |         | 세로 정렬: 0 = 글꼴 기준, 1 = 위, 2 = 가운데, 3 = 아래 |
    | FontLineHeight   | PIT_UI1 |         | 글꼴에 어울리는 줄 높이 (on/off) |
    | HeadingType      | PIT_UI1 |         | 문단 머리 모양: 0 = 없음, 1 = 개요, 2 = 번호, 3 = 불릿 |
    | Level            | PIT_UI1 |         | 단계 (0 - 6) |
    | BorderConnect    | PIT_UI1 |         | 문단 테두리/배경 - 테두리 연결 (on/off) |
    | BorderText       | PIT_UI1 |         | 문단 테두리/배경 - 여백 무시: 0 = 단, 1 = 텍스트 |
    | BorderOffsetLeft | PIT_I   |         | 문단 테두리/배경 - 4방향 간격 (HWPUNIT): 왼쪽 |
    | BorderOffsetRight| PIT_I   |         | 문단 테두리/배경 - 4방향 간격 (HWPUNIT): 오른쪽 |
    | BorderOffsetTop  | PIT_I   |         | 문단 테두리/배경 - 4방향 간격 (HWPUNIT): 위 |
    | BorderOffsetBottom| PIT_I  |         | 문단 테두리/배경 - 4방향 간격 (HWPUNIT): 아래 |
    | TailType         | PIT_UI1 |         | 문단 꼬리 모양 (마지막 꼬리 줄 적용) (on/off) |
    | LineWrap         | PIT_UI1 |         | 글꼴에 어울리는 줄 높이 (on/off) |
    | TabDef           | PIT_SET | TabDef  | 탭 정의 |
    | Numbering        | PIT_SET | NumberingShape | 문단 번호 (머리 모양이 ‘개요’, ‘번호’일 때 사용) |
    | Bullet           | PIT_SET | BulletShape | 불릿 모양 (머리 모양이 ‘불릿’일 때 사용) |
    | BorderFill       | PIT_SET | BorderFill | 테두리/배경 |
    """

    def __init__(self, parameterset):
        super().__init__(parameterset)
        self.attributes_names = [
            "left_margin",
            "right_margin",
            "indentation",
            "prev_spacing",
            "next_spacing",
            "line_spacing_type",
            "line_spacing",
            "align_type",
            "break_latin_word",
            "break_non_latin_word",
            "snap_to_grid",
            "condense",
            "widow_orphan",
            "keep_with_next",
            "keep_lines_together",
            "pagebreak_before",
            "text_alignment",
            "font_line_height",
            "heading_type",
            "level",
            "border_connect",
            "border_text",
            "border_offset_left",
            "border_offset_right",
            "border_offset_top",
            "border_offset_bottom",
            "tail_type",
            "line_wrap",
            "tab_def",
            "numbering",
            "bullet",
            "border_fill",
        ]

    left_margin = ParameterSet._int_prop("LeftMargin", "왼쪽 여백 (URC)")
    right_margin = ParameterSet._int_prop("RightMargin", "오른쪽 여백 (URC)")
    indentation = ParameterSet._int_prop("Indentation", "들여쓰기/내어 쓰기 (URC)")
    prev_spacing = ParameterSet._int_prop("PrevSpacing", "문단 간격 위 (URC)")
    next_spacing = ParameterSet._int_prop("NextSpacing", "문단 간격 아래 (URC)")
    line_spacing_type = ParameterSet._mapped_prop(
        "LineSpacingType",
        _line_spacing_type_map,
        doc="줄 간격 종류: 0 = 글꼴 기준, 1 = 고정 값, 2 = 여백만 지정",
    )
    line_spacing = ParameterSet._int_prop("LineSpacing", "줄 간격 값")
    align_type = ParameterSet._mapped_prop(
        "AlignType",
        _align_type_map,
        doc="정렬 방식: 0 = 양쪽 정렬, 1 = 왼쪽 정렬, 2 = 오른쪽 정렬, 3 = 가운데 정렬, 4 = 배분 정렬, 5 = 나눔 정렬",
    )
    break_latin_word = ParameterSet._mapped_prop(
        "BreakLatinWord",
        _latin_line_break_map,
        doc="줄 나눔 (라틴): 0 = 단어, 1 = 하이픈, 2 = 글자",
    )
    break_non_latin_word = ParameterSet._mapped_prop(
        "BreakNonLatinWord",
        _nonlatin_line_break_map,
        "줄 나눔 (비 라틴): 0 = 어절, 1 = 글자",
    )
    snap_to_grid = ParameterSet._bool_prop(
        "SnapToGrid", "편집 용지의 줄 격자 사용 (on/off)"
    )
    condense = ParameterSet._int_prop("Condense", "공백 최소값 (0 - 75%)", 0, 75)
    widow_orphan = ParameterSet._bool_prop("WidowOrphan", "외톨이줄 보호 (on/off)")
    keep_with_next = ParameterSet._bool_prop(
        "KeepWithNext", "다음 문단과 함께 (on/off)"
    )
    keep_lines_together = ParameterSet._bool_prop(
        "KeepLinesTogether", "문단 보호 (on/off)"
    )
    pagebreak_before = ParameterSet._bool_prop(
        "PagebreakBefore", "문단 앞에서 항상 쪽 나눔 (on/off)"
    )
    text_alignment = ParameterSet._mapped_prop(
        "TextAlignment",
        _text_align_map,
        doc="세로 정렬: 0 = 글꼴 기준, 1 = 위, 2 = 가운데, 3 = 아래",
    )
    font_line_height = ParameterSet._bool_prop(
        "FontLineHeight", "글꼴에 어울리는 줄 높이 (on/off)"
    )
    heading_type = ParameterSet._mapped_prop(
        "HeadingType",
        _heading_type_map,
        doc="문단 머리 모양: 0 = 없음, 1 = 개요, 2 = 번호, 3 = 불릿",
    )
    level = ParameterSet._int_prop("Level", "단계 (0 - 6)", 0, 6)
    border_connect = ParameterSet._bool_prop(
        "BorderConnect", "문단 테두리/배경 - 테두리 연결 (on/off)"
    )
    border_text = ParameterSet._mapped_prop(
        "BorderText",
        _border_text_map,
        doc="문단 테두리/배경 - 여백 무시: 0 = 단, 1 = 텍스트",
    )
    border_offset_left = ParameterSet._int_prop(
        "BorderOffsetLeft", "문단 테두리/배경 - 4방향 간격 (HWPUNIT): 왼쪽"
    )
    border_offset_right = ParameterSet._int_prop(
        "BorderOffsetRight", "문단 테두리/배경 - 4방향 간격 (HWPUNIT): 오른쪽"
    )
    border_offset_top = ParameterSet._int_prop(
        "BorderOffsetTop", "문단 테두리/배경 - 4방향 간격 (HWPUNIT): 위"
    )
    border_offset_bottom = ParameterSet._int_prop(
        "BorderOffsetBottom", "문단 테두리/배경 - 4방향 간격 (HWPUNIT): 아래"
    )
    tail_type = ParameterSet._bool_prop(
        "TailType", "문단 꼬리 모양 (마지막 꼬리 줄 적용) (on/off)"
    )
    line_wrap = ParameterSet._bool_prop("LineWrap", "글꼴에 어울리는 줄 높이 (on/off)")
    tab_def = ParameterSet._typed_prop("TabDef", "탭 정의", lambda: TabDef)
    numbering = ParameterSet._typed_prop(
        "Numbering", "문단 번호", lambda: NumberingShape
    )
    bullet = ParameterSet._typed_prop("Bullet", "불릿 모양", lambda: BulletShape)
    border_fill = ParameterSet._typed_prop(
        "BorderFill", "테두리/배경", lambda: BorderFill
    )

### Shape Object

In [ ]:
#| export
class ShapeObject(ParameterSet):
    """
    ### ShapeObject

    105) ShapeObject : 도형 개체의 속성 정의 (테이블, 수식, 그림 포함)

    이 객체는 한글의 도형 관련 속성을 조작하고 정의할 수 있으며, 도형의 크기, 위치, 스타일 등을 설정할 수 있습니다.

    | Item ID            | Type      | SubType   | Description                                   |
    |--------------------|-----------|-----------|-----------------------------------------------|
    | TreatAsChar        | PIT_UI1   |           | 글자처럼 처리 여부 (on/off)                    |
    | AffectsLine        | PIT_UI1   |           | 줄에 영향을 미치는지 여부 (on/off)             |
    | VertRelTo          | PIT_UI1   |           | 수직 기준 위치 설정                           |
    | VertAlign          | PIT_UI1   |           | 수직 정렬 방식                                |
    | VertOffset         | PIT_I4    |           | 수직 오프셋 (HWPUNIT)                          |
    | HorzRelTo          | PIT_UI1   |           | 수평 기준 위치 설정                           |
    | HorzAlign          | PIT_UI1   |           | 수평 정렬 방식                                |
    | HorzOffset         | PIT_I4    |           | 수평 오프셋 (HWPUNIT)                          |
    | FlowWithText       | PIT_UI1   |           | 텍스트 흐름에 따라 이동 여부 (on/off)           |
    | AllowOverlap       | PIT_UI1   |           | 겹침 허용 여부 (on/off)                        |
    | WidthRelTo         | PIT_UI1   |           | 너비 기준 위치 설정                           |
    | Width              | PIT_I4    |           | 도형의 너비                                   |
    | HeightRelTo        | PIT_UI1   |           | 높이 기준 위치 설정                           |
    | Height             | PIT_I4    |           | 도형의 높이                                   |
    | ProtectSize        | PIT_UI1   |           | 크기 보호 여부 (on/off)                        |
    | TextWrap           | PIT_UI1   |           | 텍스트 랩 설정                                |
    | TextFlow           | PIT_UI1   |           | 텍스트 흐름 방향                              |
    | OutsideMarginLeft  | PIT_I4    |           | 외부 여백 (왼쪽)                              |
    | OutsideMarginRight | PIT_I4    |           | 외부 여백 (오른쪽)                            |
    | OutsideMarginTop   | PIT_I4    |           | 외부 여백 (위)                                |
    | OutsideMarginBottom| PIT_I4    |           | 외부 여백 (아래)                              |
    | NumberingType      | PIT_UI1   |           | 번호 매기기 방식                              |
    | LayoutWidth        | PIT_I4    |           | 레이아웃 너비                                 |
    | LayoutHeight       | PIT_I4    |           | 레이아웃 높이                                 |
    | Lock               | PIT_UI1   |           | 잠금 여부 (on/off)                            |
    | HoldAnchorObj      | PIT_UI1   |           | 기준 객체 고정 여부                           |
    | PageNumber         | PIT_UI    |           | 페이지 번호                                   |
    | AdjustSelection    | PIT_UI1   |           | 선택 영역 조정 여부                           |
    | AdjustTextBox      | PIT_UI1   |           | 텍스트 박스 조정 여부                         |
    | AdjustPrevObjAttr  | PIT_UI1   |           | 이전 객체 속성 조정 여부                       |
    """
    def __init__(self, parameterset):
        super().__init__(parameterset)
        self.attributes_names = [
"treat_as_char", 
"affects_line", 
"vert_rel_to", 
"vert_align", 
"vert_offset", 
"horz_rel_to", 
"horz_align", 
"horz_offset", 
"flow_with_text", 
"allow_overlap", 
"width_rel_to", 
"width", 
"height_rel_to", 
"height", 
"protect_size", 
"text_wrap", 
"text_flow", 
"outside_margin_left", 
"outside_margin_right", 
"outside_margin_top", 
"outside_margin_bottom", 
"numbering_type", 
"layout_width", 
"layout_height", 
"lock", 
"hold_anchor_obj", 
"page_number", 
"adjust_selection", 
"adjust_text_box", 
"adjust_prev_obj_attr", 
"shape_draw_layout", 
"shape_draw_line_attr", 
"shape_draw_fill_attr", 
"shape_draw_image_attr", 
"shape_draw_rect_type", 
"shape_draw_arc_type", 
"shape_draw_resize", 
"shape_draw_rotate", 
"shape_draw_edit_detail", 
"shape_draw_image_scissoring", 
"shape_draw_sc_action", 
"shape_draw_ctrl_hyperlink", 
"shape_draw_coord_info", 
"shape_draw_shear", 
"shape_draw_textart", 
"shape_table_cell", 
"shape_list_properties", 
"shape_caption", 
        ]
            
    treat_as_char    = ParameterSet._bool_prop("TreatAsChar", "글자처럼 처리 여부 (on/off)")
    affects_line     = ParameterSet._bool_prop("AffectsLine", "줄에 영향을 미치는지 여부 (on/off)")
    vert_rel_to      = ParameterSet._mapped_prop("VertRelTo", _vert_rel_to_map, doc="수직 기준 위치 설정")
    vert_align       = ParameterSet._mapped_prop("VertAlign", _vert_align_map, doc="수직 정렬 방식")
    vert_offset      = ParameterSet._int_prop("VertOffset", "수직 오프셋 (HWPUNIT)")
    horz_rel_to      = ParameterSet._mapped_prop("HorzRelTo", _hroz_rel_to_map, doc="수평 기준 위치 설정")
    horz_align       = ParameterSet._mapped_prop("HorzAlign", _hroz_align_map, doc="수평 정렬 방식")
    horz_offset      = ParameterSet._int_prop("HorzOffset", "수평 오프셋 (HWPUNIT)")
    flow_with_text   = ParameterSet._bool_prop("FlowWithText", "텍스트 흐름에 따라 이동 여부 (on/off)")
    allow_overlap    = ParameterSet._bool_prop("AllowOverlap", "겹침 허용 여부 (on/off)")
    width_rel_to     = ParameterSet._mapped_prop("WidthRelTo", _hroz_rel_to_map, doc="너비 기준 위치 설정")
    width            = ParameterSet._int_prop("Width", "도형의 너비. WidthRelTo 값에 따라 의미가 달라짐(absolute는 hwpunit 나머지는 퍼센트(%))")
    height_rel_to    = ParameterSet._mapped_prop("HeightRelTo", _vert_rel_to_map, doc="높이 기준 위치 설정")
    height           = ParameterSet._int_prop("Height", "도형의 높이 HeightRelTo 값에 따라 의미가 달라짐(absolute는 hwpunit 나머지는 퍼센트(%))")
    protect_size     = ParameterSet._bool_prop("ProtectSize", "크기 보호 여부 (on/off)")
    text_wrap        = ParameterSet._mapped_prop("TextWrap", _text_wrap_map, doc="텍스트 랩 설정")
    text_flow        = ParameterSet._mapped_prop("TextFlow", _text_flow_map, doc="텍스트 흐름 방향 그리기 개체의 좌/우 어느 쪽에 글을 배치할지 지정하는 옵션. TextWrap의 값이 0일 때만 유효하다. 0 = 양쪽 모두(Both) 1 = 왼쪽만(Left Only) 2 = 오른쪽만(Right Only) 3 = 왼쪽과 오른쪽 중 넓은 쪽(Largest Only)")
    outside_margin_left  = ParameterSet._unit_prop("OutsideMarginLeft", "mili", "외부 여백 (왼쪽)")
    outside_margin_right = ParameterSet._unit_prop("OutsideMarginRight", "mili", "외부 여백 (오른쪽)")
    outside_margin_top   = ParameterSet._unit_prop("OutsideMarginTop", "mili", "외부 여백 (위)")
    outside_margin_bottom = ParameterSet._unit_prop("OutsideMarginBottom", "mili", "외부 여백 (아래)")
    numbering_type   = ParameterSet._mapped_prop("NumberingType", _numbering_type_map, doc="번호 매기기 방식")
    layout_width     = ParameterSet._int_prop("LayoutWidth", "레이아웃 너비")
    layout_height    = ParameterSet._int_prop("LayoutHeight", "레이아웃 높이")
    lock             = ParameterSet._bool_prop("Lock", "잠금 여부 (on/off)")
    hold_anchor_obj  = ParameterSet._bool_prop("HoldAnchorObj", "기준 객체 고정 여부")
    page_number      = ParameterSet._int_prop("PageNumber", "페이지 번호")
    adjust_selection = ParameterSet._bool_prop("AdjustSelection", "선택 영역 조정 여부")
    adjust_text_box  = ParameterSet._bool_prop("AdjustTextBox", "텍스트 박스 조정 여부")
    adjust_prev_obj_attr = ParameterSet._bool_prop("AdjustPrevObjAttr", "이전 객체 속성 조정 여부")

    # Composite properties using _typed_prop
    shape_draw_layout      = ParameterSet._typed_prop("ShapeDrawLayOut", "그리기 개체의 Layout", lambda: DrawLayout)
    shape_draw_line_attr   = ParameterSet._typed_prop("ShapeDrawLineAttr", "그리기 개체의 Line 속성", lambda: DrawLineAttr)
    shape_draw_fill_attr   = ParameterSet._typed_prop("ShapeDrawFillAttr", "그리기 개체의 Fill 속성", lambda: DrawFillAttr)
    shape_draw_image_attr  = ParameterSet._typed_prop("ShapeDrawImageAttr", "그림 개체 속성", lambda: DrawImageAttr)
    shape_draw_rect_type   = ParameterSet._typed_prop("ShapeDrawRectType", "사각형 그리기 개체 유형", lambda: DrawRectType)
    shape_draw_arc_type    = ParameterSet._typed_prop("ShapeDrawArcType", "호 그리기 개체 유형", lambda: DrawArcType)
    shape_draw_resize      = ParameterSet._typed_prop("ShapeDrawResize", "그리기 개체 리사이징", lambda: DrawResize)
    shape_draw_rotate      = ParameterSet._typed_prop("ShapeDrawRotate", "그리기 개체 회전", lambda: DrawRotate)
    shape_draw_edit_detail = ParameterSet._typed_prop("ShapeDrawEditDetail", "그리기 개체 EditDetail", lambda: DrawEditDetail)
    shape_draw_image_scissoring = ParameterSet._typed_prop("ShapeDrawImageScissoring", "그림 개체 자르기", lambda: DrawImageScissoring)
    shape_draw_sc_action   = ParameterSet._typed_prop("ShapeDrawScAction", "그리기 개체 회전/flip", lambda: DrawScAction)
    shape_draw_ctrl_hyperlink = ParameterSet._typed_prop("ShapeDrawCtrlHyperlink", "그리기 개체 하이퍼링크", lambda: DrawCtrlHyperlink)
    shape_draw_coord_info  = ParameterSet._typed_prop("ShapeDrawCoordInfo", "그리기 개체 좌표정보", lambda: DrawCoordInfo)
    shape_draw_shear       = ParameterSet._typed_prop("ShapeDrawShear", "그리기 개체 기울이기", lambda: DrawShear)
    shape_draw_textart     = ParameterSet._typed_prop("ShapeDrawTextart", "글맵시", lambda: DrawTextart)

    shape_table_cell       = ParameterSet._typed_prop("ShapeTableCell", "셀 정보", lambda: Cell)
    shape_list_properties  = ParameterSet._typed_prop("ShapeListProperties", "서브 list 속성", lambda: ListProperties)
    shape_caption          = ParameterSet._typed_prop("ShapeCaption", "캡션", lambda: Caption)





### TabDef

In [ ]:
#| export

class TabDef(ParameterSet):
    """
    ### TabDef

    113) TabDef : 탭 정의

    | Item ID    | Type      | SubType | Description |
    |------------|-----------|---------|-------------|
    | AutoTabLeft| PIT_UI1   |         | 문단 왼쪽 끝 탭 (on / off) |
    | AutoTabRight| PIT_UI1  |         | 문단 오른쪽 끝 탭 (on / off) |
    | TabItem    | PIT_ARRAY | PIT_I   | 각각의 탭 정의. 하나의 탭 아이템은 세 개의 인수로 표현됨. (n*3+0: 탭 위치, n*3+1: 채울 모양, n*3+2: 탭 종류) |
    """
    auto_tab_left  = ParameterSet._bool_prop("AutoTabLeft", "문단 왼쪽 탭 설정 (on / off)")
    auto_tab_right = ParameterSet._bool_prop("AutoTabRight", "문단 오른쪽 탭 설정 (on / off)")
    tab_item       = ParameterSet._int_list_prop("TabItem", "탭 정의 목록 (정수 리스트)")




### Table

In [ ]:
#| export

class Table(ParameterSet):
    """
    ### Table

    114) Table : 테이블 속성 정의

    | Item ID          | Type      | SubType         | Description                   |
    |------------------|-----------|-----------------|-------------------------------|
    | PageBreak        | PIT_UI1   |                 | 페이지 나눔 처리 방식         |
    | RepeatHeader     | PIT_UI1   |                 | 반복 헤더 여부 (on / off)      |
    | CellSpacing      | PIT_UI4   |                 | 셀 간격 (HWPUNIT)             |
    | CellMarginLeft   | PIT_I4    |                 | 셀 왼쪽 여백                 |
    | CellMarginRight  | PIT_I4    |                 | 셀 오른쪽 여백               |
    | CellMarginTop    | PIT_I4    |                 | 셀 위쪽 여백                 |
    | CellMarginBottom | PIT_I4    |                 | 셀 아래쪽 여백               |
    | BorderFill       | PIT_SET   | BorderFill      | 테두리/채우기 속성           |
    | TableCharInfo    | PIT_SET   | TableChartInfo  | 테이블 관련 문자 정보        |
    | TableBorderFill  | PIT_SET   | BorderFill      | 테이블 테두리 속성           |
    | Cell             | PIT_SET   | Cell            | 셀 정보                      |
    """
    page_break     = ParameterSet._mapped_prop("PageBreak", _page_break_map,
                                               doc="표가 페이지 경계에 걸렸을 때의 처리 방식 0 = 나누지 않는다.  1 = 테이블은 나누지만 셀은 나누지 않는다. 2 = 셀 내의 텍스트도 나눈다.")
    repeat_header  = ParameterSet._bool_prop("RepeatHeader", "반복 헤더 여부 (on/off)")
    cell_spacing   = ParameterSet._int_prop("CellSpacing", "셀 간격 (HWPUNIT)")
    cell_margin_left  = ParameterSet._int_prop("CellMarginLeft", "셀 왼쪽 여백")
    cell_margin_right = ParameterSet._int_prop("CellMarginRight", "셀 오른쪽 여백")
    cell_margin_top   = ParameterSet._int_prop("CellMarginTop", "셀 위쪽 여백")
    cell_margin_bottom = ParameterSet._int_prop("CellMarginBottom", "셀 아래쪽 여백")
    border_fill    = ParameterSet._typed_prop("BorderFill", "테두리/채우기 속성", lambda: BorderFill)
    # table_char_info = ParameterSet._typed_prop("TableCharInfo", "테이블 관련 문자 정보", TableChartInfo) # Not available now.
    table_border_fill = ParameterSet._typed_prop("TableBorderFill", "테이블 테두리 속성", lambda: BorderFill)
    cell           = ParameterSet._typed_prop("Cell", "셀 정보", lambda: Cell)



### Test 

In [ ]:
action = app.actions.CharShape
print(action.pset)

In [ ]:
action = app.actions.RepeatFind
pset = action.pset
print("값: ", pset.find_string)
pset.find_charshape.facename = "바탕"
pset.find_charshape.fonttype = 1
pset.find_string = "사월"
# pset.find_parashape.align_type = 3
# # pset.find_charshape = None
# pset.find_parashape = None
action.run()

In [ ]:
findall = app.actions.RepeatFind
findall.pset.find_charshape.facename = "바탕"
findall.pset.find_charshape.fonttype = 1

findall.pset.find_string = "사월"
findall()

In [ ]:
pset.find_charshape

In [ ]:
action = app.actions.CharShape
pset = action.pset
pset.facename_hangul

In [ ]:
pset.fonttype_hangul, pset.fonttype_hanja, pset.fonttype_japanese, pset.fonttype_latin, pset.fonttype_other, pset.fonttype_symbol, pset.fonttype_user

In [ ]:
current_charshape = CharShape(app.api.CharShape)
current_charshape.facename_hangul

In [ ]:
pset.facename_hangul = "함초롬바탕"
pset.facename_latin = "바탕"
pset.facename_user = "바탕"
pset.facename_japanese = "바탕"
pset.facename_hanja = "바탕"
pset.facename_symbol = "바탕"
# pset.facename_other = "바탕"

pset.fonttype_hangul = 1
pset.fonttype_hanja = 0
pset.fonttype_japanese = 0 
pset.fonttype_latin = 0
pset.fonttype_other = 0
pset.fonttype_symbol = 0 
pset.fonttype_user = 0 
# pset.height = 10000
action.run()

In [ ]:
action = app.actions.RepeatFind
pset = action._get_hset()
pset.FindCharShape.FontTypeHanja = 0
pset.FindCharShape.FontTypeJapanese = 0
pset.FindCharShape.FontTypeLatin = 0
pset.FindCharShape.FontTypeOther = 0
pset.FindCharShape.FontTypeSymbol = 0
pset.FindCharShape.FontTypeUser = 0 
action.run(pset)

In [ ]:
pset.FindString
pset.FindCharShape.FaceNameHangul
pset.FindCharShape.FontTypeHangul
pset.FindCharShape.FontTypeLatin

In [ ]:
pset.FindCharShape.FaceNameHangul = "함초롬바탕"
action.run()

In [ ]:
app.api.HParameterSet.HFindReplace.FindCharShape.FaceNameHangul

In [ ]:
app.api.HParameterSet.HFindReplace.FindCharShape.FaceNameHangul = "함초롬바탕"

### 중요한 사항

 - "찾기 기능"에서 글자 모양은 모든 글자가 동일해야 동작한다.
 - 다를 경우에는 작동하지 않는 것으로 추정된다.

In [ ]:
cs = CharShape()
cs.facename="바탕"

app.find_text(
    "사월", 
    charshape=cs,
    )